In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Notebook author:** emeinhardt@ucsd.edu

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Requirements" data-toc-modified-id="Requirements-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Requirements</a></span></li><li><span><a href="#Usage" data-toc-modified-id="Usage-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Usage</a></span></li></ul></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Imports-/-load-data" data-toc-modified-id="Imports-/-load-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports / load data</a></span><ul class="toc-item"><li><span><a href="#Load/extract-sanity-checking-data" data-toc-modified-id="Load/extract-sanity-checking-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load/extract sanity-checking data</a></span><ul class="toc-item"><li><span><a href="#Segmental-lexicon,-prefixes,-inventory,-and-triphones" data-toc-modified-id="Segmental-lexicon,-prefixes,-inventory,-and-triphones-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Segmental lexicon, prefixes, inventory, and triphones</a></span></li><li><span><a href="#Triphone-channel-distribution-and-channel-alphabet" data-toc-modified-id="Triphone-channel-distribution-and-channel-alphabet-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Triphone channel distribution and channel alphabet</a></span></li><li><span><a href="#Preview-and-postview-distributions-and-alphabets" data-toc-modified-id="Preview-and-postview-distributions-and-alphabets-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Preview and postview distributions and alphabets</a></span></li><li><span><a href="#Corpus-contexts" data-toc-modified-id="Corpus-contexts-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Corpus contexts</a></span></li><li><span><a href="#Conversion-to-one-hot-vectors-/-sequences-thereof" data-toc-modified-id="Conversion-to-one-hot-vectors-/-sequences-thereof-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Conversion to one-hot vectors / sequences thereof</a></span></li></ul></li><li><span><a href="#Load-segmental-sequence-channel-matrices-$p(Y_0^k|X_0^k)$-and-$p(Y_0^f|X_0^f)$" data-toc-modified-id="Load-segmental-sequence-channel-matrices-$p(Y_0^k|X_0^k)$-and-$p(Y_0^f|X_0^f)$-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Load segmental sequence channel matrices $p(Y_0^k|X_0^k)$ and $p(Y_0^f|X_0^f)$</a></span></li><li><span><a href="#Load-contextual-distribution-on-segmental-wordforms-$p(W|C)$" data-toc-modified-id="Load-contextual-distribution-on-segmental-wordforms-$p(W|C)$-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load contextual distribution on segmental wordforms $p(W|C)$</a></span></li><li><span><a href="#Load-lexicon-metadata" data-toc-modified-id="Load-lexicon-metadata-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Load lexicon metadata</a></span></li></ul></li><li><span><a href="#Slow,-but-sanity-checking-calculations" data-toc-modified-id="Slow,-but-sanity-checking-calculations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Slow, but sanity-checking calculations</a></span><ul class="toc-item"><li><span><a href="#$p(C_0^i-|-X_0^i;-X_{i+1})$" data-toc-modified-id="$p(C_0^i-|-X_0^i;-X_{i+1})$-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>$p(C_0^i | X_0^i; X_{i+1})$</a></span></li><li><span><a href="#$p(Y_0^i|X_0^k)$" data-toc-modified-id="$p(Y_0^i|X_0^k)$-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>$p(Y_0^i|X_0^k)$</a></span></li><li><span><a href="#$p(X_0^f|C)$" data-toc-modified-id="$p(X_0^f|C)$-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>$p(X_0^f|C)$</a></span></li><li><span><a href="#$p(Y_0^i-|-c)$" data-toc-modified-id="$p(Y_0^i-|-c)$-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>$p(Y_0^i | c)$</a></span></li><li><span><a href="#$p(Y_0^i|x_0^k)$-sampler" data-toc-modified-id="$p(Y_0^i|x_0^k)$-sampler-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>$p(Y_0^i|x_0^k)$ sampler</a></span></li><li><span><a href="#$p(\widehat{X_0^f}|Y_0^{i},-C)$" data-toc-modified-id="$p(\widehat{X_0^f}|Y_0^{i},-C)$-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>$p(\widehat{X_0^f}|Y_0^{i}, C)$</a></span></li><li><span><a href="#$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1},-C)$" data-toc-modified-id="$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1},-C)$-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1}, C)$</a></span></li></ul></li><li><span><a href="#Definition-of-core-calculation" data-toc-modified-id="Definition-of-core-calculation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Definition of core calculation</a></span><ul class="toc-item"><li><span><a href="#Channel-sampler" data-toc-modified-id="Channel-sampler-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Channel sampler</a></span></li><li><span><a href="#Sample-parallel-posterior-estimator" data-toc-modified-id="Sample-parallel-posterior-estimator-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Sample-parallel posterior estimator</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Testing</a></span></li><li><span><a href="#Calculation-volume-organization-for-Case-1" data-toc-modified-id="Calculation-volume-organization-for-Case-1-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Calculation volume organization for Case 1</a></span><ul class="toc-item"><li><span><a href="#Slices:-Calculate-results-for-groups-of-reconstructed-wordforms-by-(input-prefix,-context)" data-toc-modified-id="Slices:-Calculate-results-for-groups-of-reconstructed-wordforms-by-(input-prefix,-context)-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>Slices: Calculate results for groups of reconstructed wordforms by (input prefix, context)</a></span></li><li><span><a href="#Blocks:-Calculate-results-for-groups-of-input-prefixes-by-context" data-toc-modified-id="Blocks:-Calculate-results-for-groups-of-input-prefixes-by-context-5.4.2"><span class="toc-item-num">5.4.2&nbsp;&nbsp;</span>Blocks: Calculate results for groups of input prefixes by context</a></span></li></ul></li><li><span><a href="#Calculation-volume-organization-2" data-toc-modified-id="Calculation-volume-organization-2-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Calculation volume organization 2</a></span></li><li><span><a href="#Calculation-volume-organization-3" data-toc-modified-id="Calculation-volume-organization-3-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Calculation volume organization 3</a></span><ul class="toc-item"><li><span><a href="#Slices:-Calculate-$\{p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)\}$-for-all-$W$-and-for-a-single-given-$c$" data-toc-modified-id="Slices:-Calculate-$\{p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)\}$-for-all-$W$-and-for-a-single-given-$c$-5.6.1"><span class="toc-item-num">5.6.1&nbsp;&nbsp;</span>Slices: Calculate $\{p(\widehat{W} = w^* | W = w^*, C = c)\}$ for all $W$ and for a single given $c$</a></span></li></ul></li></ul></li><li><span><a href="#Calculate-distribution" data-toc-modified-id="Calculate-distribution-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Calculate distribution</a></span><ul class="toc-item"><li><span><a href="#Number-of-calculations-necessary-under-different-calculation-criteria" data-toc-modified-id="Number-of-calculations-necessary-under-different-calculation-criteria-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Number of calculations necessary under different calculation criteria</a></span></li><li><span><a href="#Determine-dimensions-and-cells-to-be-calculated-vs.-ignored" data-toc-modified-id="Determine-dimensions-and-cells-to-be-calculated-vs.-ignored-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Determine dimensions and cells to be calculated vs. ignored</a></span><ul class="toc-item"><li><span><a href="#Option:-Only-calculate-full-source-wordforms-rather-than-all-source-prefixes?" data-toc-modified-id="Option:-Only-calculate-full-source-wordforms-rather-than-all-source-prefixes?-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Option: Only calculate full source wordforms rather than all source prefixes?</a></span></li><li><span><a href="#Suboption:-Given-$\neg-r$-and-a-full-source-wordform-$w^*$,-only-calculate-posterior-probability-$p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)$,-$\forall-w^*$?" data-toc-modified-id="Suboption:-Given-$\neg-r$-and-a-full-source-wordform-$w^*$,-only-calculate-posterior-probability-$p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)$,-$\forall-w^*$?-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Suboption: Given $\neg r$ and a full source wordform $w^*$, only calculate posterior probability $p(\widehat{W} = w^* | W = w^*, C = c)$, $\forall w^*$?</a></span></li><li><span><a href="#Suboption:-Restrict-calculations-only-to-(source-sequence,-reconstruction-sequence)-pairs-within-edit-distance-$k$?" data-toc-modified-id="Suboption:-Restrict-calculations-only-to-(source-sequence,-reconstruction-sequence)-pairs-within-edit-distance-$k$?-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Suboption: Restrict calculations only to (source sequence, reconstruction sequence) pairs within edit distance $k$?</a></span></li><li><span><a href="#Altogether..." data-toc-modified-id="Altogether...-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Altogether...</a></span></li></ul></li><li><span><a href="#Case-1:-$r-\land-(0-<-k-\leq-4)$" data-toc-modified-id="Case-1:-$r-\land-(0-<-k-\leq-4)$-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Case 1: $r \land (0 &lt; k \leq 4)$</a></span></li><li><span><a href="#Case-2:-$\neg-r-\land-(0-<-k-\leq-4)-\land-\neg-e$" data-toc-modified-id="Case-2:-$\neg-r-\land-(0-<-k-\leq-4)-\land-\neg-e$-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Case 2: $\neg r \land (0 &lt; k \leq 4) \land \neg e$</a></span></li><li><span><a href="#Case-3:-$\neg-r-\land-e$" data-toc-modified-id="Case-3:-$\neg-r-\land-e$-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Case 3: $\neg r \land e$</a></span></li></ul></li></ul></div>

# Overview

Given a choice of parameters $\epsilon$ and $n$, and given
 - wordform channel matrices $p(Y_0^f|X_0^f)$
 - a contextual distribution on segmental wordforms $p(X_0^f|C)$
 - segmental lexicon metadata pre-calculating $k$-cousins/$k$-spheres up to $k=4$
 
Calculate

$$\hat{p}(\hat{X}_0^f = x_0^{'f}|X_0^f = x_0^{*f}, c) = \frac{1}{n} \sum\limits_{y_0^f \in S} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c)$$
 where 
  - edit distance $d(x_0^{'f}, x_0^{*f}) \leq 4$
  - $S = $ a set of $n$ samples from $p(Y_0^f|x_0^{*f})$. In practice an $n \approx 50$ seems to result in estimates that are within $10^{-6}$ of the true estimate. 
  - $p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) = \frac{p(y_0^f|x_0^{'f})p(x_0^{'f}|c)}{p(y_0^f | c)}$
  - $p(y_0^f| c) = \sum\limits_{v', x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|v')p(v'|c) = \sum\limits_{x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|c)$

## Requirements

#FIXME

## Usage

#FIXME

# Parameters

In [2]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

In [3]:
from boilerplate import *

In [4]:
repo_dir = getcwd(); repo_dir

'/mnt/cube/home/AD/emeinhar/wr'

In [94]:
# Parameters

#p(Y_0^k|X_0^k) - from step 4d
c = ''
# c = 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_CMs_by_length_by_prefix_index.pickle'
c = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_CMs_by_length_by_prefix_index.pickle'

#p(Y_0^f|X_0^f) - from step 4d
h = ''
h = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_exact_CMs_by_length_by_wordform_index.pickle'

#p(X_0^f|C) - from step 4c
w = ''
# w = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_C.npy'
w = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_C.npy'

# LTR metadata directory - contains pre-computed k-cousin/k-sphere information
m = ''
# m = 'LTR_Buckeye_aligned_w_GD_AmE_destressed'
m = 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed'

# output filepath prefix for pW_WC
o = ''
# o = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_WC'
# o = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC'
o = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts'

# for sanity checking...

# the sorted prefixes of W here correspond to the W dimension of file 'w'
p = ''
# p = 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json'
p = 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json'

# the sorted conditioning triphones and outcome response uniphones correspond to
# the dimensions of file 'c'
t = ''
# t = 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'
t = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'

# the sorted projected contexts here correspond to the C dimension of file 'w'
s = ''
# s = 'LD_Fisher_vocab_in_Buckeye_contexts/LM_filtered_buckeye_contexts.txt'
s = 'LD_Fisher_vocab_in_swbd2003_contexts/LM_filtered_swbd2003_contexts.txt'

# calculation parameters
n = ''
n = '50'

k = ''
k = '2'

r = ''
r = 'False' #if 'False', only calculate p(\hat{W}|W = w, c), i.e. don't calculate p(\hat{W}|P = p, c)

e = ''
e = 'True' #if r='False' and e='True', only calculate p(\hat{W} = w*| W = w*, c) ∀w ∈ W

In [6]:
output_dir = path.dirname(o)
if not path.exists(output_dir):
    print('Making output path {0}'.format(output_dir))
    makedirs(output_dir)

In [95]:
if n == '':
    n = 50
else:
    n = int(n)

In [96]:
if k == '':
    k = 2
else:
    k = int(k)

In [97]:
if r == '' or r == 'False':
    r = False
#     raise Exception('Assuming only full wordforms have been produced is not supported currently.')
elif r == 'True':
    r = True
else:
    raise Exception(f"r must be one of {'','True','False'}, got '{r}' instead")

In [98]:
if not r and (e == '' or e == 'True'):
    e = True
#     raise Exception('Only calculating the posterior probability of the actual (correct) full intended wordform is not supported currently.')
elif r and (e == '' or e == 'True'):
    raise Exception("e can only be True if r is False")
elif e == 'False':
    e = False
else:
    raise Exception(f"e must be one of {'','True','False'}, got '{e}'")

In [11]:
# postview_fp = path.join(path.dirname(t), "p6Y0X01.json"); postview_fp

# preview_fp = path.join(path.dirname(t), "p3Y1X01.json"); preview_fp

# Imports / load data

In [12]:
import pickle

In [13]:
import numpy as np
import torch
import sparse

In [14]:
import tiledb

In [15]:
from probdist import *

In [16]:
from tqdm import tqdm, tqdm_gui, tqdm_notebook

In [17]:
from joblib import Parallel, delayed

# J = -1
J = 16
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def identity(x):
    return x

def par(gen_expr):
    return Parallel(n_jobs=J, backend=BACKEND, verbose=V, prefer=PREFER)(gen_expr)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
    if torch.cuda.device_count() > 1:
        print(torch.cuda.get_device_name(1))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(1)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(1)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2070
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
GeForce RTX 2070
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [19]:
gpu = torch.device('cuda')
cpu = torch.device('cpu')

my_device = cpu

In [20]:
cuda_ft = torch.cuda.FloatTensor
cuda_dt = torch.cuda.DoubleTensor

ft = torch.FloatTensor
dt = torch.DoubleTensor

my_ft = ft
my_dt = dt

# torch.set_default_tensor_type(my_ft)
torch.set_default_tensor_type(my_dt)

## Load/extract sanity-checking data

We want to be able to make queries, spot checks, and sanity checks. That means we want to be able to reference
 1. the set of strings constituting segmental wordforms and prefixes
 2. the source and channel alphabets
 3. the channel distribution's conditioning triphones ∩ the triphones in the lexicon = the triphones in the lexicon
 4.  

Segmental wordforms were necessary for 
 - the lexicon metadata calculation (step 4b)
 - the contextual distribution on segmental wordforms (step 4c)
 - the definition of the segmental sequence channel matrices (step 4d)
 
What does each use as input? (Pass that to this notebook.)

Each notebook uses ...pW_V.json (or something slightly downstream of that)

### Segmental lexicon, prefixes, inventory, and triphones

In [21]:
pW_V = condDistsAsProbDists(importProbDist(p))

Extract 
 - `orthographic vocabulary`
 - `segmental vocabulary`

In [22]:
Vs = set(pW_V.keys())
Ws = union(mapValues(lambda dist: set(conditions(dist)), 
                     pW_V).values())
len(Vs)
len(Ws)

9411

9172

In [23]:
wordlengthsInclEdges = set(len(ds2t(w)) for w in Ws)
wordlengthsInclEdges
numWordsOfExactlyLength = {l:len(wordformsOfLength(l, Ws, True)) for l in wordlengthsInclEdges}
numWordsOfExactlyLength

{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18}

{3: 5,
 4: 173,
 5: 1266,
 6: 1790,
 7: 1646,
 8: 1328,
 9: 1052,
 10: 828,
 11: 508,
 12: 329,
 13: 150,
 14: 66,
 15: 24,
 16: 6,
 18: 1}

In [145]:
wordLengthRange = sorted(list(range(min(wordlengthsInclEdges),
                                    max(wordlengthsInclEdges)+1)))
wordLengthRange

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [24]:
wordlengthsNotIncludingEdges = {each-2 for each in wordlengthsInclEdges}
wordlengthsNotIncludingEdges

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16}

In [25]:
lengthFreqs = {l:len(wordformsAtLeastLlong(l, Ws, True)) for l in wordlengthsInclEdges}
lengthFreqs

{3: 9172,
 4: 9167,
 5: 8994,
 6: 7728,
 7: 5938,
 8: 4292,
 9: 2964,
 10: 1912,
 11: 1084,
 12: 576,
 13: 247,
 14: 97,
 15: 31,
 16: 7,
 18: 1}

Create mappings between `segmental wordforms` and `segmental prefixes`.

In [26]:
prefix_relation = set(union({(w,p) for p in getPrefixes(w)} for w in tqdm(Ws)))
len(prefix_relation)

100%|██████████| 9172/9172 [00:18<00:00, 508.32it/s]  


70555

Extract 
 - `segmental prefixes`

In [27]:
Ps = set(map(lambda pair: pair[1],
             prefix_relation))

Create 
 - `sorted` versions of the `segmental vocabulary` and `segmental prefixes`

In [28]:
Ws_t = tuple(sorted(list(Ws)))
Ps_t = tuple(sorted(list(Ps)))
num_wordforms = len(Ws_t)
num_prefixes = len(Ps_t)

num_wordforms
num_prefixes

9172

33059

Create mappings between `orthographic wordforms` and `segmental wordforms`.

In [29]:
v_to_Ws = mapValues(lambda dist: set(conditions(dist)),
                    pW_V)
V_W_relation = {(v,w) 
                for v in v_to_Ws 
                for w in v_to_Ws[v]}
w_to_Vs = {w:{v for v in Vs if (v,w) in V_W_relation}
           for w in Ws}

Extract
 - `segmental inventory`
 - `triphones` in the `segmental lexicon`

In [30]:
source_alphabet = lexiconToInventory(Ws)
Xs = source_alphabet
len(Xs)
Xs

41

{'aɪ',
 'aʊ',
 'b',
 'd',
 'dʒ',
 'eɪ',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ',
 'p',
 's',
 't',
 'tʃ',
 'u',
 'v',
 'w',
 'z',
 'æ',
 'ð',
 'ŋ',
 'ɑ',
 'ɔɪ',
 'ə',
 'ɚ',
 'ɛ',
 'ɪ',
 'ɹ',
 'ʃ',
 'ʊ',
 'ʌ',
 'ʒ',
 'θ',
 '⋉',
 '⋊'}

In [31]:
Xs_t = tuple(sorted(Xs))

In [32]:
lexicon_triphones = lexiconTo3factors(Ws)
len(lexicon_triphones)
list(lexicon_triphones)[:10]

7381

['l.ɪ.ŋ',
 'ɹ.v.⋉',
 'f.æ.k',
 'h.æ.f',
 's.p.ɚ',
 '⋊.f.ɪ',
 't.ʌ.m',
 'ʌ.b.ɚ',
 'ɑ.k.ɚ',
 'k.oʊ.l']

In [33]:
X012s = lexicon_triphones
X012s_t = tuple(sorted(list(X012s)))

In [34]:
triphs_with_LE = {triph for triph in lexicon_triphones if leftEdge in triph}
len(triphs_with_LE)

489

In [35]:
triphs_with_RE = {triph for triph in lexicon_triphones if rightEdge in triph}
len(triphs_with_RE)

510

In [36]:
lexicon_diphones = lexiconTo2factors(Ws)
len(lexicon_diphones)
list(lexicon_diphones)[:10]

1002

['oʊ.k', 'v.⋉', 'v.n', 'g.p', 'dʒ.ʌ', 'θ.d', '⋊.u', 'k.u', 'ɑ.n', 'oʊ.ɪ']

In [37]:
X01s = lexicon_diphones
X01s_t = tuple(sorted(list(X01s)))

In [38]:
X01s_with_LE = {diph for diph in X01s if leftEdge in diph}
len(X01s_with_LE)

36

In [39]:
X01s_with_RE = {diph for diph in X01s if rightEdge in diph}
len(X01s_with_RE)

31

### Triphone channel distribution and channel alphabet

In [40]:
p3Y1X012 = condDistsAsProbDists(importProbDist(t))

assert uniformOutcomes(p3Y1X012)

In [41]:
channel_triphones = conditions(p3Y1X012)
len(channel_triphones)

assert all(triph in channel_triphones for triph in lexicon_triphones)

7381

In [42]:
channel_alphabet = outcomes(p3Y1X012)
len(channel_alphabet)
channel_alphabet

38

{'aɪ',
 'aʊ',
 'b',
 'd',
 'dʒ',
 'eɪ',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ',
 'p',
 's',
 't',
 'tʃ',
 'u',
 'v',
 'w',
 'z',
 'æ',
 'ð',
 'ŋ',
 'ɑ',
 'ɔɪ',
 'ɚ',
 'ɛ',
 'ɪ',
 'ɹ',
 'ʃ',
 'ʊ',
 'ʌ',
 'ʒ',
 'θ'}

In [43]:
Y1s = channel_alphabet
Y1s_t = tuple(sorted(Y1s))

In [44]:
in_lexicon_inventory_but_not_in_channel_inventory = source_alphabet - channel_alphabet
in_lexicon_inventory_but_not_in_channel_inventory

assert in_lexicon_inventory_but_not_in_channel_inventory == {leftEdge, rightEdge} or in_lexicon_inventory_but_not_in_channel_inventory == {'ə', leftEdge, rightEdge}

{'ə', '⋉', '⋊'}

### Preview and postview distributions and alphabets

In [45]:
# preview_fp
# postview_fp

In [46]:
# p3Y1X01 = condDistsAsProbDists(importProbDist(preview_fp))

# p6Y0X01 = condDistsAsProbDists(importProbDist(postview_fp))

In [47]:
# assert uniformOutcomes(p3Y1X01)

# assert uniformOutcomes(p6Y0X01)

In [48]:
# preview_outcomes = outcomes(p3Y1X01)

# postview_outcomes = outcomes(p6Y0X01)

# assert preview_outcomes == Y1s

# assert postview_outcomes == Y1s

In [49]:
# preview_channel_diphones = conditions(p3Y1X01)
# len(preview_channel_diphones)

# # missing_preview_conditions = {diph for diph in lexicon_diphones if (diph not in preview_channel_diphones) and (leftEdge not in diph)}
# # missing_preview_conditions

# # p3Y1X01 = condProbDistAsDicts(p3Y1X01)

# # #add 𝚺⋉ to preview conditions
# # # for each 𝛔⋉ ∈ 𝚺⋉, ensure p(Y_1 = ⋉|𝛔⋉) = 1.0
# # for c in missing_preview_conditions:
# #     p3Y1X01[c] = {o:1.0 if ds2t(o)[1] == rightEdge else 0.0 for o in Y1s}

# # # add ⋉ to preview outcomes
# # for c in preview_channel_diphones
    
# # p3Y1X01 = condDistsAsProbDists(p3Y1X01)

# assert all(diph in preview_channel_diphones for diph in lexicon_diphones if not (leftEdge in diph or rightEdge in diph))

In [50]:
# postview_channel_diphones = conditions(p6Y0X01)
# len(postview_channel_diphones)

# assert all(diph in postview_channel_diphones for diph in lexicon_diphones if not (leftEdge in diph or rightEdge in diph))

In [51]:
# p3Y1X01 = condProbDistAsDicts(p3Y1X01)
# p6Y0X01 = condProbDistAsDicts(p6Y0X01)

# p3Y1X01 = {c:p3Y1X01[c] for c in lexicon_diphones if not (leftEdge in c or rightEdge in c)}
# p6Y0X01 = {c:p6Y0X01[c] for c in lexicon_diphones if not (leftEdge in c or rightEdge in c)}

# assert areNormalized(p3Y1X01)
# assert areNormalized(p6Y0X01)
# assert uniformOutcomes(p3Y1X01)
# assert uniformOutcomes(p6Y0X01)

# p3Y1X01 = condDistsAsProbDists(p3Y1X01)
# p6Y0X01 = condDistsAsProbDists(p6Y0X01)

In [52]:
# len(conditions(p3Y1X01))
# len(conditions(p6Y0X01))

In [53]:
# Y012s = set(map(t2ds, sigmaK(set.union(Y1s, edgeSymbols), 3)))
# len(set.union(Y1s, edgeSymbols))
# len(Y012s)
# Y012s_t = tuple(sorted(list(Y012s)))

### Corpus contexts

In [54]:
Cs = importSeqs(s, list)
len(Cs)

Cs_t = tuple(sorted(Cs))
Cs_t[123:129]

assert tuple(Cs) == Cs_t

106295

('<rem> and doctor',
 '<rem> and electric',
 '<rem> and especially',
 '<rem> and even',
 '<rem> and for',
 '<rem> and he')

### Conversion to one-hot vectors / sequences thereof

In [55]:
# construct what you need to convert to/from one-hot representations

# look at segment sequence channel matrix notebook

In [56]:
Xmap = seqsToIndexMap(Xs)
XOHmap = seqsToOneHotMap(Xs)

In [57]:
X012map = seqsToIndexMap(X012s)
# X012OHs = seqMapToOneHots(X012map)
X012OHmap = seqsToOneHotMap(X012s)

In [58]:
Y1map = seqsToIndexMap(Y1s)

In [59]:
def dsToUniphoneIndices(ds, uniphoneToIndexMap):
    uniphoneSeq = ds2t(ds)
    return np.array([uniphoneToIndexMap[uniphone] for uniphone in uniphoneSeq])

def dsToUniphoneOHs(ds, uniphoneToOHmap):
    uniphoneSeq = ds2t(ds)
    return np.array([uniphoneToOHmap[uniphone] for uniphone in uniphoneSeq])

def dsToTriphoneSeq(ds):
    return dsToKfactorSequence(3, ds)

def dsToTriphoneIndices(ds, triphoneToIndexMap):
    triphoneSeq = dsToTriphoneSeq(ds)
    return np.array([triphoneToIndexMap[triphone] for triphone in triphoneSeq])

def dsToTriphoneOHs(ds, triphoneToOHmap):
    triphoneSeq = dsToTriphoneSeq(ds)
    return np.array([triphoneToOHmap[triphone] for triphone in triphoneSeq])

dsToUniphoneIndices('t.i.f.l', Xmap)
dsToUniphoneOHs('t.i.f.l', XOHmap)
dsToTriphoneSeq('t.i.f.l')
dsToTriphoneIndices('t.i.f.l', X012map)
dsToTriphoneOHs('t.i.f.l', X012OHmap)
dsToTriphoneOHs('t.i.f.l', X012OHmap).shape
dsToTriphoneOHs('t.i.f.l', X012OHmap)[0].shape
dsToTriphoneOHs('t.i.f.l', X012OHmap)[0][5528]
dsToTriphoneOHs('t.i.f.l', X012OHmap)[1][5352]

array([18,  9,  6, 12])

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.]])

('t.i.f', 'i.f.l')

array([3740, 1441])

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

(2, 7381)

(7381,)

0.0

0.0

In [60]:
# Y1s_RE = outcomes(p3Y1X01)
# len(Y1s_RE)
# Y1s_RE_list = sorted(list(Y1s_RE))

# print(Y1s_RE - Y1s)

# Y1REmap = seqsToIndexMap(Y1s_RE)

# Y1REOHs = seqMapToOneHots(Y1REmap)
# Y1REOHmap = seqsToOneHotMap(Y1s_RE)
Y1OHmap = seqsToOneHotMap(Y1s)
# OHY1REmap = oneHotToSeqMap(Y1s_RE)
OHY1map = oneHotToSeqMap(Y1s)

def ymap(y):
#     return Y1REmap[y]
    return Y1map[y]

def y0kMap(y0k):
    return np.array(list(map(ymap, ds2t(y0k)[1:])))

def channelSeqOHs2ds(y0k_OHs, addLeftEdge = False):
    if not addLeftEdge:
#         return t2ds(tuple( map(OHY1REmap, tuple(y0k_OHs) ) ))
        return t2ds(tuple( map(OHY1map, tuple(y0k_OHs) ) ))
#     return leftEdge + '.' + t2ds(tuple( map(OHY1REmap, tuple(y0k_OHs) ) ))
    return leftEdge + '.' + t2ds(tuple( map(OHY1map, tuple(y0k_OHs) ) ))

def channelSeqds2OHs(y0k):
    y0k_t = ds2t(y0k)
    if leftEdge == y0k_t[0]:
        y1k_t = y0k_t[1:]
    y1k_t = y0k_t[1:]
#     return np.array([Y1REOHmap[ yj ] for yj in y1k_t]) #shape should be (_, 39)
    return np.array([Y1OHmap[ yj ] for yj in y1k_t]) #shape should be (_, 38)

def y0kOHmap(y0k):
##     y0k_t = ds2t(y0k)
## #     y0k_indices = y0kMap(y0k) #np.array(list(map(lambda y1: Y1map[y1], y0k_t[1:])))
##     y1k = t2ds(y0k_t[1:])
##     y0k_OHs = dsToUniphoneOHs(y1k, Y1REOHmap)
#     return dsToUniphoneOHs(t2ds(ds2t(y0k)[1:]), Y1REOHmap)
    y0k_t = ds2t(y0k) #let l = len(y0k_t)
    if y0k_t[0] == leftEdge:
        y1k_t = y0k_t[1:]
#         return np.array([Y1REOHmap[ yj ] for yj in y1k_t]) #shape should be (l-1, 39)
        return np.array([Y1OHmap[ yj ] for yj in y1k_t]) #shape should be (l-1, 38)
#     return np.array([Y1REOHmap[ yj ] for yj in y0k_t]) #shape should be (l, 39)
    return np.array([Y1OHmap[ yj ] for yj in y0k_t]) #shape should be (l, 38)


# list(Y1s_RE)[0]
list(Y1s)[0]
# OHY1REmap( Y1REOHmap[ list(Y1s_RE)[0] ] )
OHY1map( Y1OHmap[ list(Y1s)[0] ] )

channelSeqds2OHs(leftEdge + '.'+ 't.u.p')

channelSeqOHs2ds( channelSeqds2OHs(leftEdge + '.'+ 't.u.p') , True)

channelSeqOHs2ds(y0kOHmap(leftEdge + '.' + 't.u.p'), True)
channelSeqOHs2ds(y0kOHmap('t.u.p'))
len(ds2t(leftEdge + '.' + 't.u.p'))
len(ds2t('t.u.p'))
y0kOHmap('t.u.p').shape
y0kOHmap(leftEdge + '.' + 't.u.p').shape

'u'

'u'

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

'⋊.t.u.p'

'⋊.t.u.p'

't.u.p'

4

3

(3, 38)

(3, 38)

In [61]:
p3Y1X012_np = condDistFamilyToNP(p3Y1X012)
p3Y1X012_np.shape

(38, 7381)

In [62]:
from random import choice

In [63]:
random_source_wordform = choice(list(Ws))
random_source_wordform

random_source_prefix = choice(list(Ps))
random_source_prefix

'⋊.ɹ.ɪ.k.u.p.ɚ.eɪ.t.⋉'

'⋊.f.ɪ.l.ɪ'

In [64]:
def randomPrefix(l, alphabet=Xs):
    return randomString(alphabet, l, hasLeftEdge=True)

In [65]:
random_channel_prefix2 = randomPrefix(len(ds2t(random_source_wordform))-1, alphabet=Y1s)
random_channel_prefix2

'⋊.p.ɹ.j.ɛ.v.m.w.θ.s'

In [66]:
# random_source_prefix = getRandomKey(pX0i)
random_source_prefix = choice(list(Ps))
while ds2t(random_source_prefix)[-1] == rightEdge:
#     random_source_prefix = getRandomKey(pX0i)
    random_source_prefix = choice(list(Ps))
while len(ds2t(random_source_prefix)) > len(ds2t(random_source_wordform)):
#     random_source_prefix = getRandomKey(pX0i)
    random_source_prefix = choice(list(Ps))
random_source_prefix
random_channel_prefix = randomPrefix(len(ds2t(random_source_prefix))-1, alphabet=Y1s)
random_channel_prefix

'⋊.æ.k.t.ɪ.v.ə.t'

'⋊.θ.f.w.dʒ.ð.h.s'

In [67]:
def sourcePrefixToTriphones(x0k):
    xp_t = ds2t(x0k) #"x prefix"
    
#     xi = xp_t[-2] #just-completed segment
#     xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    
#     xik_ds = t2ds((xi, xk))
#     preview_dist = p3Y1X01[xik_ds]
    
    x012s = dsToKfactorSequence(3, t2ds(xp_t))
    return x012s

random_triphoneSeq = sourcePrefixToTriphones(random_source_prefix)
random_triphoneSeq
threeFactorSequenceToDS(random_triphoneSeq)

('⋊.æ.k', 'æ.k.t', 'k.t.ɪ', 't.ɪ.v', 'ɪ.v.ə', 'v.ə.t')

'⋊.æ.k.t.ɪ.v.ə.t'

In [68]:
def sourcePrefixToTriphoneIndices(x0k):
    triphoneSequence = sourcePrefixToTriphones(x0k)
    return tuple(map(lambda x012: X012map[x012], triphoneSequence))

sourcePrefixToTriphoneIndices(random_source_prefix)

(7235, 4618, 1808, 3938, 6036, 4287)

## Load segmental sequence channel matrices $p(Y_0^k|X_0^k)$ and $p(Y_0^f|X_0^f)$

Index $i$ of `pY0k_X0ks` is a tensor of shape $|W| \times |Y_1| \times (i-2)$: for each wordform $w$ in $W$, there is a length $i-2$ stack of vectors giving the channel distribution for the length $i$ prefix (including the left word edge symbol and the upcoming segment) of $w$. Where necessary and appropriate (viz. when the wordform's length is incongruent w.r.t. the channel string), a wordform's channel matrix has been padded with columns of 0s to reflect the fact that the relevant source prefix could never have given rise to a channel string of that length.

`pY0f_X0fs` is an analogous tensor with the same shape, but intended for use in contexts where only wordforms whose length exactly matches the length of a channel string is worth considering as a cause of the channel string.

In [99]:
pY0k_X0ks = pickle.load(open(c, 'rb'))
len(pY0k_X0ks)

19

In [101]:
pY0f_X0fs = pickle.load(open(h, 'rb'))
len(pY0f_X0fs)

19

In [102]:
pY0k_X0ks[0].dtype
pY0f_X0fs[0].dtype

dtype('float64')

dtype('float64')

In [103]:
wordlengthsNotIncludingEdges
wordlengthsInclEdges
lengthFreqs

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16}

{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18}

{3: 9172,
 4: 9167,
 5: 8994,
 6: 7728,
 7: 5938,
 8: 4292,
 9: 2964,
 10: 1912,
 11: 1084,
 12: 576,
 13: 247,
 14: 97,
 15: 31,
 16: 7,
 18: 1}

In [104]:
len(Ws_t)
len(Y1s_t)

9172

38

In [105]:
pY0k_X0ks[0].shape
pY0k_X0ks[1].shape
pY0k_X0ks[2].shape
pY0k_X0ks[3].shape

pY0f_X0fs[0].shape
pY0f_X0fs[1].shape
pY0f_X0fs[2].shape
pY0f_X0fs[3].shape

(0, 0)

(0, 0)

(0, 0)

(9172, 38, 1)

(0, 0)

(0, 0)

(0, 0)

(9172, 38, 1)

In [106]:
pY0k_X0ks[3][0]
pY0f_X0fs[3][0]

array([[ 0.65264711],
       [ 0.00700978],
       [ 0.02545656],
       [ 0.02545656],
       [ 0.00682531],
       [ 0.06235012],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.0051651 ],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00664084],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978],
       [ 0.00700978],
       [ 0.00682531],
       [ 0.00700978]])

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [107]:
p3Y1X012_np.shape

(38, 7381)

In [108]:
pY0k_X0ks[3].shape
pY0f_X0fs[3].shape

(9172, 38, 1)

(9172, 38, 1)

In [109]:
def retrieve_CM_for(x0f_WE, total_prefix_length, exact = False):
    '''
    total_prefix_length includes 
     - the "⋊" symbol,
     - the actually and completely produced prefix,
     - the next (i.e. upcoming) segment.
     
    Ex:
      w = '⋊.aɪ.d.i.l.aɪ.z.⋉'
      |'⋊.aɪ.d.i.l.aɪ.z.⋉'| = 8
      l = 5
    =>
      'total_prefix' = '⋊.aɪ.d.i.l'
      |'⋊.aɪ.d.i.l'| = 5
      The associated channel matrix is of shape (|Y1s|, l-2 = 5-2 = 3).
    '''
    if not exact:
        return pY0k_X0ks[total_prefix_length][Ws_t.index(x0f_WE)]
    else:
        return pY0f_X0fs[total_prefix_length][Ws_t.index(x0f_WE)]

In [110]:
w_0 = Ws_t[0]; w_0
w_0_produced_prefix_noLE = t2ds(ds2t(Ws_t[0])[1:3+1]); w_0_produced_prefix_noLE, len(ds2t(w_0_produced_prefix_noLE))
w_0_produced_prefix = t2ds(ds2t(Ws_t[0])[0:3+1]); w_0_produced_prefix, len(ds2t(w_0_produced_prefix))
w_0_produced_prefix_plus_next = t2ds(ds2t(Ws_t[0])[0:3+2]); w_0_produced_prefix_plus_next, len(ds2t(w_0_produced_prefix_plus_next))

pY0k_X0ks[5][0].shape
# pY0k_X0ks[3][0] #assume the produced prefix is of length i+1=3+1=4 (not including left word edge)
pY0k_X0ks[5][0] # assume the produced prefix is of length i-2=3 (not including left word edge)
                # assume the produced prefix is of length i-1=4 (including left word edge)
                # assume the produced prefix + upcoming segment is of length i=5 (including left word edge)

w_0_whole_prefix = t2ds(ds2t(Ws_t[0])[0:3+2]); w_0_whole_prefix
w_0_whole_prefix_3factors = sourcePrefixToTriphones(w_0_whole_prefix); w_0_whole_prefix_3factors
w_0_whole_prefix_3factor_indices = sourcePrefixToTriphoneIndices(w_0_whole_prefix); w_0_whole_prefix_3factor_indices
w_0_prefix_CM = np.vstack([p3Y1X012_np[:,source_triph_idx] for source_triph_idx in w_0_whole_prefix_3factor_indices]).T; w_0_prefix_CM
w_0_prefix_CM.shape

assert np.allclose( w_0_prefix_CM, pY0k_X0ks[5][0] )
assert np.allclose( pY0k_X0ks[5][0], retrieve_CM_for(Ws_t[0], 5))

'⋊.aɪ.d.i.l.aɪ.z.⋉'

('aɪ.d.i', 3)

('⋊.aɪ.d.i', 4)

('⋊.aɪ.d.i.l', 5)

(38, 3)

...

'⋊.aɪ.d.i.l'

('⋊.aɪ.d', 'aɪ.d.i', 'd.i.l')

(6892, 5, 495)

array([[ 0.65264711,  0.02141693,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.02545656,  0.00855777,  0.00703146],
       [ 0.02545656,  0.58122414,  0.00693832],
       [ 0.00682531,  0.00821919,  0.00675326],
       [ 0.06235012,  0.00855777,  0.00703146],
       [ 0.00682531,  0.00833257,  0.00684642],
       [ 0.00700978,  0.02552273,  0.00703146],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.00855777,  0.73787478],
       [ 0.0051651 ,  0.00734595,  0.00603577],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00844442,  0.00693832],
       [ 0.00700978,  0.01630829,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00844442,  0.00693832],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.05787536,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00855777,  0.007

(38, 3)

**Notes:**

 - `|pY0k_X0ks|` = max_l = |w|. ∄w'. |w'| > |w| (around 16-18, usually) 
 - `pY0k_X0ks[i]` = the lexical channel matrices for 'total prefixes' of length `i` (i.e. incl. word edges and upcoming segment)
 - `pY0k_X0ks[i]` =  the lexical channel matrices for prefixes of length `i-1` (i.e. *not* incl. word edges, but incl. upcoming segment)
 - `|pY0k_X0ks[i]|` = `(|W|, |Y1|, i-2)` = (segmental wordforms, Y1s, number of triphones)
 
(The same statements hold wrt `pY0fX0f`.)

## Load contextual distribution on segmental wordforms $p(W|C)$

In [111]:
pW_C = np.load(w)
pW_C.shape
pW_C.dtype
pW_C.nbytes / 1e9

(9172, 106295)

dtype('float64')

7.79950192

In [112]:
len(Ws_t)
len(Cs_t)

9172

106295

In [113]:
Cs_t[Cs_t.index('a couple of')]
pW_C[:,Cs_t.index('a couple of')]
isNormalized_np(pW_C[:,Cs_t.index('a couple of')])

'a couple of'

array([  2.52036749e-10,   1.50634197e-08,   2.52036749e-10, ...,
         5.54693797e-10,   4.15967189e-08,   6.22123621e-07])

True

In [114]:
Cs_t.index('a couple of')

1753

In [115]:
def pW_C_lookup(w=None,c=None):
    if w is None and c is None:
        raise Exception('Must specify at least one of a context string or segmental wordform string')
    if w is None:
        my_pW_c = pW_C[:,Cs_t.index(c)]
        my_pW_c_as_dict = dict(zip(Ws_t, my_pW_c))
        assert isNormalized(my_pW_c_as_dict)
        return ProbDist(my_pW_c_as_dict)
    if w is not None and c is not None:
        my_pw_c = pW_C[Ws_t.index(w), Cs_t.index(c)]
        return my_pw_c
    if c is None:
        my_pw_C = pW_C[Ws_t.index(w), :]
        my_pw_C_as_dict = dict(zip(Cs_t, my_pw_C))
        return my_pw_C_as_dict

In [116]:
my_dist = pW_C_lookup(c='a couple of')

In [117]:
my_dist.most_common(10)

[('⋊.t.aɪ.m.z.⋉', 0.70170688240694257),
 ('⋊.ð.ɛ.m.⋉', 0.058586970250377776),
 ('⋊.ð.ə.⋉', 0.054670419056498991),
 ('⋊.ʌ.ð.ɚ.⋉', 0.036645416028036247),
 ('⋊.m.aɪ.⋉', 0.028159696907432537),
 ('⋊.ð.oʊ.z.⋉', 0.020634293629461325),
 ('⋊.h.ʌ.n.d.ɹ.ə.d.⋉', 0.017489202057292059),
 ('⋊.l.aɪ.k.⋉', 0.014006091693604115),
 ('⋊.g.ʊ.d.⋉', 0.005842215226429354),
 ('⋊.m.ɪ.l.i.ə.n.⋉', 0.0053049068571393984)]

In [118]:
random_context = choice(Cs_t); random_context
Cs_t.index(random_context)
pW_C_lookup(c=random_context).most_common(10)

"it's surprising with"

40209

[('⋊.s.ʌ.m.⋉', 0.60962073668761252),
 ('⋊.ð.ə.⋉', 0.078899524235941362),
 ('⋊.ə.⋉', 0.037216534342601304),
 ('⋊.m.aɪ.⋉', 0.022202704882751199),
 ('⋊.ð.æ.t.⋉', 0.02208662471885002),
 ('⋊.ð.ɛ.m.⋉', 0.018828581041691678),
 ('⋊.j.u.⋉', 0.018759395570309208),
 ('⋊.ɪ.t.⋉', 0.018226097481862602),
 ('⋊.m.i.⋉', 0.015706837474504534),
 ('⋊.h.ɚ.⋉', 0.011288497185342564)]

## Load lexicon metadata

In [119]:
cousin_fn_map = {i:'{0}cousinsOf.npz'.format(i) 
                 for i in range(5)}
sphere_fn_map = {i:'{0}spheresOf.npz'.format(i) 
                 for i in range(5)}

In [120]:
cousin_fn_map

{0: '0cousinsOf.npz',
 1: '1cousinsOf.npz',
 2: '2cousinsOf.npz',
 3: '3cousinsOf.npz',
 4: '4cousinsOf.npz'}

In [121]:
sphere_fn_map

{0: '0spheresOf.npz',
 1: '1spheresOf.npz',
 2: '2spheresOf.npz',
 3: '3spheresOf.npz',
 4: '4spheresOf.npz'}

In [122]:
assert all(fn in listdir(m) for fn in cousin_fn_map.values())
assert all(fn in listdir(m) for fn in sphere_fn_map.values())

In [123]:
chdir(m)

In [124]:
cousin_mats = mapValues(sparse.load_npz, cousin_fn_map)

In [125]:
sphere_mats = mapValues(sparse.load_npz, sphere_fn_map)

In [126]:
chdir(repo_dir)

The $k$th-cousin/sphere matrix $m$ is a matrix of shape $|P| \times |W|$, where $m_{i,j}$ = 1 iff prefix $p_i$ is in a $k$-cousin/sphere relation to wordform $w_j$.

In [127]:
len(Ps_t)
len(Ws_t)

33059

9172

In [128]:
cousin_mats
mapValues(lambda m: m.nbytes / 1e9,
          cousin_mats)

{0: <COO: shape=(33059, 9172), dtype=uint8, nnz=70555, fill_value=0>,
 1: <COO: shape=(33059, 9172), dtype=uint8, nnz=966262, fill_value=0>,
 2: <COO: shape=(33059, 9172), dtype=uint8, nnz=8845129, fill_value=0>,
 3: <COO: shape=(33059, 9172), dtype=uint8, nnz=36334546, fill_value=0>,
 4: <COO: shape=(33059, 9172), dtype=uint8, nnz=59184373, fill_value=0>}

{0: 0.001199435,
 1: 0.016426454,
 2: 0.150367193,
 3: 0.617687282,
 4: 1.006134341}

In [129]:
sphere_mats
mapValues(lambda m: m.nbytes / 1e9,
          sphere_mats)

{0: <COO: shape=(33059, 9172), dtype=uint8, nnz=9172, fill_value=0>,
 1: <COO: shape=(33059, 9172), dtype=uint8, nnz=30652, fill_value=0>,
 2: <COO: shape=(33059, 9172), dtype=uint8, nnz=370148, fill_value=0>,
 3: <COO: shape=(33059, 9172), dtype=uint8, nnz=2062370, fill_value=0>,
 4: <COO: shape=(33059, 9172), dtype=uint8, nnz=3207990, fill_value=0>}

{0: 0.000155924, 1: 0.000521084, 2: 0.006292516, 3: 0.03506029, 4: 0.05453583}

In [130]:
# c1.coords

In [131]:
c1 = cousin_mats[1]
(c1.coords[0][-2], c1.coords[1][-2])
Ps_t[c1.coords[0][-2]]
Ws_t[c1.coords[1][-2]]

(33057, 6949)

'⋊.θ.ʌ.n.d.ɚ'

'⋊.w.ʌ.n.d.ɚ.⋉'

In [132]:
m

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed'

In [ ]:
# segmental_wordforms = importSeqs(path.join(m, 'LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V_Transcriptions.txt'))
# len(segmental_wordforms)

In [ ]:
# list(segmental_wordforms)[0]

In [ ]:
# Ws_t = tuple(sorted(list(map(padInputSequenceWithBoundaries,
#                              segmental_wordforms))))
# len(Ws_t)

In [ ]:
# Ps = union(map(lambda w: getPrefixes(padInputSequenceWithBoundaries(w)), segmental_wordforms))
# len(Ps)

In [ ]:
# Ps_t = tuple(sorted(list(Ps)))
# len(Ps_t)

# Slow, but sanity-checking calculations

In [ ]:
# FIXME update this cell once you add in cells for doing calculations the old-fashioned way 
# and verifying/demonstrating that the calculation cell works correctly

## $p(C_0^i | X_0^i; X_{i+1})$

$C_i = (Y_{i-1}^{x_i}, Y_i^{x_i}, Y_{i+1}^{x_i})$

$p(y_{i-1}, y_i; y_{i+1} | x_{i-1}^i; x_{i+1}) = p(y_{i-1} | x_{i-1}, x_i;) p(y_i | x_{i-1}^i ; x_{i+1}) p(y_{i+1} | x_i ; x_{i+1})$

$p(c_0^i | x_0^i; x_{i+1}) = \prod\limits_{j=0}^{j=i} p(c_i | x_{i-1}^i ; x_{i+1})$

In [ ]:
# def pC1_X012(y012, x012):
#     x012_t = ds2t(x012)
#     y012_t = ds2t(y012)
    
#     x0, x1, x2 = x012_t[0], x012_t[1], x012_t[2]
#     x01 = (x0, x1)
#     x12 = (x1, x2)
    
#     y0, y1, y2 = y012_t[0], y012_t[1], y012_t[2]
    
#     if x0 == leftEdge:
#         if y0 == leftEdge:
#             left_term = 1.0
#         else:
#             left_term = 0.0
#     else:
#         if y0 == leftEdge:
#             left_term = 0.0
#         else:
#             left_term = p6Y0X01[x01][y0]
    
#     center_term = p3Y1X012[x012][y1]
    
#     if x1 == rightEdge:
#         if y1 == rightEdge:
#             right_term = 1.0
#         else:
#             right_term = 0.0
#     else:
#         if y1 == rightEdge:
#             right_term = 0.0
#         else:
#             right_term = p3Y1X01[x12][y2]
    
#     terms = (left_term, center_term, right_term)
    
#     result = prod(terms)
    
#     return result

# def pC0iX0k(c0i, x0k):
#     xp_t = ds2t(x0k)
#     three_factor_seq = dsToKfactorSequence(3, x0k)
    
#     inputs = zip(c0i, three_factor_seq)
    
#     terms = starmap(pC1_X012, inputs)
    
#     result = prod(terms)
    
#     return result

In [ ]:
# s = leftEdge + '.i.p.i.' + rightEdge; s
# dsToKfactorSequence(3, s)
# threeFactorSequenceToDS(dsToKfactorSequence(3, s))

In [ ]:
# len(Y012s)
# len(X012s)
# len(Y012s) * len(X012s)

In [ ]:
# # takes 5.5m on sidious and 45GB
# square = set(product(Y012s, X012s))

In [ ]:
# def pY012_X012_calc(y012, x012):
#     return (x012, pC1_X012(y012, x012))

# pY012_X012 = dict(par(delayed(pY012_X012_calc)(y012, x012) for y012, x012 in product(Y012s, X012s)))

# # def pY012_X012_calc(y012_x012_pair):
# #     y012, x012 = y012_x012_pair
# #     return (x012, pC1_X012(y012, x012))

# # pY012_X012 = dict(par(delayed(pY012_X012_calc)(y012_x012_pair) for y012_x012_pair in product(Y012s, X012s)))

## $p(Y_0^i|X_0^k)$

In [143]:
len(pY0k_X0ks)
sorted(wordlengthsInclEdges)
assert len(pY0k_X0ks)-1 == sorted(wordlengthsInclEdges)[-1]

19

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18]

In [144]:
len(Ws)
len(Ps)

9172

33059

In [139]:
pY0k_X0ks[3].shape

(9172, 38, 1)

In [136]:
def pY0iX0k(y0i, x0k, debug = False):
    xp_t = ds2t(x0k) #"x prefix"
    yp_t = ds2t(y0i) #"y prefix"
#     assert xp_t[0] == leftEdge
#     assert yp_t[0] == leftEdge
    if len(xp_t) < 3:
        raise Exception('|x0k| must be ≥ 3.')
    if len(yp_t)+1 != len(xp_t):
        raise Exception('|y0i| must = |x0k|-1.')
#     if len(yp_t) != len(xp_t):
#         raise Exception('Lengths of y0i and x0k must match.')
    # if len(xp_t) == 1:
    #     if xp_t[0] != leftEdge:
    #         raise Exception('First symbol of x0k must be left word edge symbol.')    
    #     if yp_t[0] == leftEdge:
    #         return 1.0
    #     else:
    #         return 0.0
    
#     xi = xp_t[-2] #just-completed segment
#     xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    # yk = yp_t[-1]
    # xik_ds = t2ds((xi, xk))
    # preview_dist = p3Y1X01[xik_ds]
#     if yk not in preview_dist:
#         print('y0i: {0}\nx0k: {1}\nxi: {2}\nxk: {3}\nyk: {4}\nxik: {5}'.format(y0i, x0k, xi, xk, yk, xik_ds))
#     preview_term = preview_dist[yk]
    
#     if debug:
#         print('xi = {0}\nxk = {1}\nyk = {2}'.format(xi, xk, yk))
#         print('preview term = {0}'.format(preview_term))
    
#     x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
    # y0i_t = yp_t[:-1]
#     x0i_ds = t2ds(x0i_t)
#     y0i_ds = t2ds(y0i_t)

#     if debug:
#         x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
#         x0i_ds = t2ds(x0i_t)
#         y0i_ds = t2ds(y0i_t)
#         print('produced prefix x0i = {0}'.format(x0i_ds))
#         print('perceived prefix y0i = {0}'.format(y0i_ds))
    
#     x012s = dsToKfactorSequence(3, x0i_ds)
#     x012s = dsToKfactorSequence(3, t2ds(xp_t))
    x012s = dsToKfactorSequence(3, x0k)
    # y1s = y0i_t[1:]
    y1s = yp_t[1:]
#     pairs = list(zip(x012s, y1s))
#     produced_terms = tuple([p3Y1X012[x012][y1] for x012, y1 in pairs])
    pairs = zip(x012s, y1s)
    produced_terms = (p3Y1X012[x012][y1] for x012, y1 in pairs)
    
#     result = preview_term * prod(produced_terms)
    result = prod(produced_terms)

#     if debug:
#         print('x012s: {0}'.format(x012s))
#         print('y1s: {0}'.format(y1s))
#         print('main terms: {0}'.format(produced_terms))
#         print('end result = {0}'.format(result))
    
    if debug:
#         return result, [p3Y1X012[x012] for x012 in x012s]
        return result, np.array([[p3Y1X012[x012][k] for k in sorted(p3Y1X012[x012].keys())] for x012 in x012s]).T

    return result

In [146]:
w_0
w_0_prefix
w_0_3 = t2ds(ds2t(w_0)[:3+2]); w_0_3 + 1
w_0_4 = t2ds(ds2t(w_0)[:4+2]); w_0_4 + 1
pY0k_X0ks[5][0]

'⋊.aɪ.d.i.l.aɪ.z.⋉'

'aɪ.d.i.l'

'⋊.aɪ.d.i.l'

'⋊.aɪ.d.i.l.aɪ'

array([[ 0.65264711,  0.02141693,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.02545656,  0.00855777,  0.00703146],
       [ 0.02545656,  0.58122414,  0.00693832],
       [ 0.00682531,  0.00821919,  0.00675326],
       [ 0.06235012,  0.00855777,  0.00703146],
       [ 0.00682531,  0.00833257,  0.00684642],
       [ 0.00700978,  0.02552273,  0.00703146],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.00855777,  0.73787478],
       [ 0.0051651 ,  0.00734595,  0.00603577],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00844442,  0.00693832],
       [ 0.00700978,  0.01630829,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00844442,  0.00693832],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.05787536,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00855777,  0.007

In [150]:
pY0iX0k(w_0_3, w_0_4)
pY0iX0k(w_0_3, w_0_4, True)[1]
assert np.allclose( pY0k_X0ks[6][0], pY0iX0k(w_0_3, w_0_4, True)[1] )

0.1983147788326225

array([[ 0.65264711,  0.02141693,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.58122414,  0.00693832,  0.00713029],
       [ 0.00682531,  0.00821919,  0.00675326,  0.00694011],
       [ 0.06235012,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00833257,  0.00684642,  0.00703584],
       [ 0.00700978,  0.02552273,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.73787478,  0.007226  ],
       [ 0.0051651 ,  0.00734595,  0.00603577,  0.00620276],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.00703146,  0.70851712],
       [ 0.00700978,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.01630829,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.02155085],
       [ 0.00700978,  0.

In [151]:
pY0iX0k('⋊.k.ɑ.n', '⋊.k.ɑ.n.t')
pY0iX0k('⋊.k.ɑ.m', '⋊.k.ɑ.n.t')

0.25068286913932425

0.0034938697256087312

In [152]:
def trimToMatch(y0i, x0f):
    x0f_t = ds2t(x0f)
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if len(x0f_t) < l+1:
        raise Exception('|x0f| must ≥ |y0i| + 1.')
    x0k_t = x0f_t[:l+1]
    x0k = t2ds(x0k_t)
    return x0k

def trimToLength(l, x0f, pad=False, padSymbol='?'):
    x0f_t = ds2t(x0f)
#     y0i_t = ds2t(y0i)
#     l = len(y0i_t)
    if len(x0f_t) < l and not pad:
        raise Exception('x0f must be at least as long as l.')
    if len(x0f_t) < l and pad:
        x0k_t = x0f_t[:l]
        x0k_t_padded = tuple( list(x0k_t) + [padSymbol] * (l - len(x0k_t)) )
        return t2ds(x0k_t_padded)
    else:
        x0k_t = x0f_t[:l]
        x0k = t2ds(x0k_t)
        return x0k
    
# trimToMatch('t.u.p', 't.u.p')
trimToMatch('t.u.p', 't.u.p.l')
trimToLength(3, 't.u.p')
trimToLength(3, 't.u.p.l')
trimToLength(6, 't.u.p.l', True)

't.u.p.l'

't.u.p'

't.u.p'

't.u.p.l.?.?'

In [153]:
def pY0iX0f(y0i, x0f, debug = False):
    x0f_t = ds2t(x0f)
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if len(x0f_t) < l+1:
        raise Exception('|x0f| must ≥ |y0i| + 1')
    x0k_t = x0f_t[:l+1]
    x0k = t2ds(x0k_t)
#     print(y0i)
#     print(x0k)
    return pY0iX0k(y0i, x0k, debug)

In [154]:
random_x0f = choice(Ws_t); random_x0f
lw = len(ds2t(random_x0f))
while lw-1 <= 2:
    random_x0f = getRandomKey(pW); random_x0f
    lw = len(ds2t(random_x0f))

random_y0i = randomPrefix(choice(range(2,lw-1)), alphabet=Y1s); random_y0i
random_x0f
random_y0i
pY0iX0f(random_y0i, random_x0f)

'⋊.ð.aɪ.⋉'

'⋊.p.θ'

'⋊.ð.aɪ.⋉'

'⋊.p.θ'

4.7843892524650475e-05

## $p(X_0^f|C)$

In [155]:
pW_C.shape
(len(Ws), len(Cs))
assert pW_C.shape[0] == len(Ws)
assert pW_C.shape[1] == len(Cs)

(9172, 106295)

(9172, 106295)

In [156]:
Cs_t[Cs_t.index('a couple of')]

pX0f = pW_C[:,Cs_t.index('a couple of')]

pW = pW_C_lookup(c = Cs_t[Cs_t.index('a couple of')])

'a couple of'

In [157]:
def pX0f_C(w, c):
    return pW_C[:, Cs_t.index(c)]

## $p(Y_0^i | c)$

In [158]:
def pXhat0fY0i_unnormalized(xhat0f, y0i, c):
    likelihood = pY0iX0f(y0i, xhat0f)
#     prior = pW[xhat0f]
    prior = pW_C_lookup(xhat0f, c)
    return likelihood * prior

def pY0i(y0i, c):
#     l = len(ds2t(y0i))
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if y0i_t[-1] == rightEdge:
        possible_source_wordforms = {w for w in Ws if len(ds2t(w)) == l}
    else:
        possible_source_wordforms = {w for w in Ws if len(ds2t(w)) >= l+1}
#     x0ksWithLengthl = prefixesWithLength(l)
#     py0kx0ks = tuple(pXhat0fY0k_unnormalized(x0k, y0k) for x0k in x0ksWithLengthl)

#     sufficiently_long_words = {w for w in Ws if len(ds2t(w)) >= l+1}
#     print(len(sufficiently_long_words))
    #line below is slower by about a third (pypy/kotoba)
#     sufficiently_long_words = wordformsAtLeastLlong(l, True)

    py0ix0fs = (pXhat0fY0i_unnormalized(x0f, y0i, c) for x0f in possible_source_wordforms)
#     def foo(x0f):
#         return pXhat0fY0k_unnormalized(x0f, y0k)
    # there's no apparent benefit to using foo + generator
#     py0kx0fs = (foo(x0f) for x0f in sufficiently_long_words)
# Parallel(n_jobs=-1, verbose=50, batch_size=4, prefer="processes", backend="multiprocessing")
#     py0kx0fs = Parallel(n_jobs=8, batch_size=4, verbose=50, prefer="threads", backend="multiprocessing")(delayed(pXhat0fY0k_unnormalized)(x0f, y0k) for x0f in sufficiently_long_words)
#     py0kx0fs = Parallel(n_jobs=8, prefer="threads")(delayed(pXhat0fY0k_unnormalized)(x0f, y0k) for x0f in sufficiently_long_words)
    return sum(py0ix0fs)

In [159]:
random_context = choice(Cs_t); random_context
pY0i('⋊.k.ɑ.n.t', random_context)

"we've got bill"

3.8087186973938132e-06

In [160]:
{w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}

{'⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉',
 '⋊.k.ɑ.n.t.n.ɪ.n.t.⋉',
 '⋊.k.ɑ.n.t.æ.k.t.⋉',
 '⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉',
 '⋊.k.ɑ.n.t.ɛ.k.s.t.⋉',
 '⋊.k.ɑ.n.t.ɛ.s.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉'}

In [161]:
random_context = choice(Cs_t); random_context
pXhat0fY0i_unnormalized('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context)

'the it would'

5.8152864416345394e-09

## $p(Y_0^i|x_0^k)$ sampler

In [162]:
def sample_pY0iX0k(x0k, debug = False):
    xp_t = ds2t(x0k) #"x prefix"
#     yp_t = ds2t(y0i) #"y prefix"
#     if len(yp_t) != len(xp_t):
#         raise Exception('Lengths of y0i and x0i must match.')
    if len(xp_t) == 1:
        if xp_t[0] != leftEdge:
            raise Exception('First symbol of x0k must be left word edge symbol.')
        else:
            return leftEdge
    if len(xp_t) < 3:
        raise Exception('|x0k| must be ≥ 3')
    
    # xi = xp_t[-2] #just-completed segment
    # xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    # xik = t2ds((xi, xk))
    # yk = sampleFrom(p3Y1X01[xik])
#     yk = yp_t[-1]
#     preview_term = p3Y1X01[t2ds((xi, xk))][yk]
    
#     if debug:
#         print('xi = {0}\nxk = {1}\nyk = {2}'.format(xi, xk, yk))
#         print('preview term = {0}'.format(preview_term))
    
#     x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
#     y0i_t = yp_t[:-1]
#     x0i_ds = t2ds(x0i_t)
#     y0i_ds = t2ds(y0i_t)

#     if debug:
#         print('produced prefix x0i = {0}'.format(x0i_ds))
#         print('perceived prefix y0i = {0}'.format(y0i_ds))
    
    x012s = dsToKfactorSequence(3, t2ds(xp_t))
#     print('x012s: {0}'.format(x012s))
#     y1s = y0i_t[1:]
    y1s = [sampleFrom(p3Y1X012[x012]) for x012 in x012s]
#     print('y1s: {0}'.format(y1s))
#     pairs = list(zip(x012s, y1s))
#     produced_terms = tuple([p3Y1X012[x012][y1] for x012, y1 in pairs])
    y0i_t = tuple( [leftEdge] + y1s )
#     print('y0i_t = {0}'.format(y0i_t))
    y0i_ds = t2ds(y0i_t)
#     print('y0i_ds = {0}'.format(y0i_ds))
#     assert len(y0i_t) == len(x0i_t)
    # yp_t = tuple([leftEdge] + y1s + [yk])
    # y0i = t2ds(yp_t)
    if debug:
#         sorted_outcomes = Y1s_t
        return y0i_ds, np.array([[p3Y1X012[x012][k] for k in Y1s_t] for x012 in x012s]).T
#         return y0i_ds, np.array([[p3Y1X012[x012][k] for k in sorted(p3Y1X012[x012].keys())] for x012 in x012s]).T
#         print('x012s: {0}'.format(x012s))
#         print('y1s: {0}'.format(tuple(y1s)))
#         print('main terms: {0}'.format(produced_terms))

    return y0i_ds
#     return preview_term * prod(produced_terms)

In [163]:
random_source_prefix
random_channel_samples = [sample_pY0iX0k(random_source_prefix) for each in range(50)]; random_channel_samples

'⋊.æ.k.t.ɪ.v.ə.t'

['⋊.b.eɪ.t.ʌ.dʒ.aɪ',
 '⋊.ɚ.k.t.ɪ.v.ʊ',
 '⋊.ɑ.ʒ.t.oʊ.v.l',
 '⋊.aɪ.k.t.ɪ.v.tʃ',
 '⋊.ʌ.ɪ.dʒ.ð.ɑ.ɪ',
 '⋊.aɪ.eɪ.f.ɪ.v.ɛ',
 '⋊.æ.ʌ.m.ɪ.v.ʌ',
 '⋊.ɑ.k.t.ɑ.t.i',
 '⋊.ʒ.aɪ.t.ʃ.t.ɔɪ',
 '⋊.æ.ɚ.t.ŋ.v.aɪ',
 '⋊.æ.oʊ.w.ʃ.v.h',
 '⋊.ɑ.n.t.ɪ.v.s',
 '⋊.æ.ʌ.t.b.v.ɔɪ',
 '⋊.æ.d.ʃ.ɪ.v.j',
 '⋊.æ.k.k.ʊ.v.ʌ',
 '⋊.s.k.t.ɪ.v.m',
 '⋊.aɪ.eɪ.aɪ.ɪ.v.eɪ',
 '⋊.æ.k.t.ʊ.t.ʌ',
 '⋊.æ.h.t.ɪ.ŋ.ʌ',
 '⋊.ɑ.f.t.i.v.u',
 '⋊.aɪ.eɪ.t.ɪ.ð.ʌ',
 '⋊.æ.eɪ.m.k.v.g',
 '⋊.æ.b.ɚ.f.oʊ.v',
 '⋊.ɑ.k.t.ɪ.v.ʃ',
 '⋊.ɑ.k.u.f.v.b',
 '⋊.aɪ.k.t.ɪ.v.tʃ',
 '⋊.ɑ.k.æ.ɪ.m.eɪ',
 '⋊.ɪ.b.t.f.i.l',
 '⋊.æ.k.θ.ɪ.ʃ.ʌ',
 '⋊.tʃ.w.t.w.v.l',
 '⋊.ɑ.i.h.ɪ.v.s',
 '⋊.θ.z.t.ɪ.v.n',
 '⋊.n.ɪ.t.ɪ.v.eɪ',
 '⋊.ɑ.ɪ.t.ɪ.ʒ.eɪ',
 '⋊.aɪ.k.t.ɪ.v.ʌ',
 '⋊.ɑ.θ.ɔɪ.ɪ.ð.ʌ',
 '⋊.æ.k.t.ɪ.v.u',
 '⋊.aɪ.k.t.v.ɛ.ɪ',
 '⋊.ɑ.k.ʃ.ɪ.v.eɪ',
 '⋊.aɪ.h.t.ɪ.v.b',
 '⋊.aɪ.m.f.ɪ.v.ʒ',
 '⋊.æ.k.t.ɑ.v.ʌ',
 '⋊.æ.k.n.ɪ.z.ɑ',
 '⋊.æ.k.t.ɪ.v.aɪ',
 '⋊.ɑ.k.t.ɪ.v.eɪ',
 '⋊.æ.g.ɔɪ.s.v.ɹ',
 '⋊.aɪ.eɪ.n.aʊ.v.ɑ',
 '⋊.j.k.t.ɪ.ɛ.p',
 '⋊.b.t.t.ɪ.ʒ.ð',
 '⋊.g.k.t.ʊ.v.ɛ']

In [164]:
[sample_pY0iX0k('⋊.k.ɑ.n.t') for each in range(50)]

['⋊.k.ɑ.z',
 '⋊.ʌ.ɑ.n',
 '⋊.aɪ.ɑ.f',
 '⋊.k.ɑ.ŋ',
 '⋊.aɪ.ɑ.ɹ',
 '⋊.k.ɹ.w',
 '⋊.b.ɑ.n',
 '⋊.k.p.f',
 '⋊.k.ɑ.l',
 '⋊.ʌ.ɑ.n',
 '⋊.k.u.ɪ',
 '⋊.b.ɑ.n',
 '⋊.p.ɹ.n',
 '⋊.k.ɑ.n',
 '⋊.b.ɔɪ.ɪ',
 '⋊.eɪ.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ʌ.n',
 '⋊.k.ʌ.n',
 '⋊.d.aɪ.n',
 '⋊.k.ɑ.l',
 '⋊.k.ɑ.n',
 '⋊.t.z.n',
 '⋊.l.ɑ.s',
 '⋊.k.ʌ.t',
 '⋊.b.ɑ.n',
 '⋊.b.ɑ.p',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.ɔɪ.ɑ.n',
 '⋊.ʒ.z.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.oʊ.f',
 '⋊.k.ɑ.ɛ',
 '⋊.k.ɑ.n',
 '⋊.i.v.ɪ',
 '⋊.k.ɑ.n',
 '⋊.k.k.g',
 '⋊.m.ɑ.d',
 '⋊.ɚ.ʌ.n',
 '⋊.k.n.n',
 '⋊.b.ʌ.n',
 '⋊.k.ɑ.ð',
 '⋊.k.ʒ.n',
 '⋊.k.ɑ.aɪ',
 '⋊.n.ɑ.ɹ',
 '⋊.ɪ.ɑ.n',
 '⋊.k.z.n']

In [165]:
sample_pY0iX0k('⋊.k.ɑ.n.t', debug=True)[1].shape
sample_pY0iX0k('⋊.k.ɑ.n.t', debug=True)[1]

(38, 3)

array([[ 0.04390334,  0.01805178,  0.00911065],
       [ 0.00700978,  0.01374583,  0.00911065],
       [ 0.02545656,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00682531,  0.00692774,  0.01700733],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00682531,  0.0070233 ,  0.0088709 ],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00682531,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.0051651 ,  0.00619171,  0.00782054],
       [ 0.65246265,  0.00711758,  0.00898997],
       [ 0.02545656,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.6450247 ],
       [ 0.00700978,  0.01374583,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00682531,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00721312,  0.009

## $p(\widehat{X_0^f}|Y_0^{i}, C)$

$p(\widehat{x_0^f}|y_0^{i}, c) = \frac{p(y_0^{i} | \widehat{x_0^f})p(\widehat{x_0^f} | c)}{p(y_0^{i})}$

In [166]:
def pXhat0fY0i_old(xhat0f, y0i, c):
    numerator = pXhat0fY0i_unnormalized(xhat0f, y0i, c)
    denominator = pY0i(y0i, c)
    if denominator == 0.0:
#         if numerator == 0.0:
#             print('both numerator and denominator are 0')
#         else:
#             print('just the denominator is 0')
        return 0
    return numerator / denominator

random_context = choice(Cs_t); random_context
(random_source_wordform, random_channel_prefix, random_context)
pXhat0fY0i_old(random_source_wordform, random_channel_prefix, random_context)

'the experience i'

('⋊.ɹ.ɪ.k.u.p.ɚ.eɪ.t.⋉', '⋊.θ.f.w.dʒ.ð.h.s', 'the experience i')

1.9802388950046044e-09

## $\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1}, C)$

$p(\widehat{x_0^f} | x_0^i;x_{i+1}, c) = \sum\limits_{y_0^{i}} p(\widehat{x_0^f} | y_0^{i}, c)p(y_0^{i}|x_0^{i};x_{i+1}, c)$

$\widehat{p}(\widehat{x_0^f}|x_0^i;x_{i+1}, c) = \frac{1}{m}\sum\limits_{m \text{ sampled } y_0^{i}\text{s from } p(Y_0^{i} | x_0^{i};x_{i+1})} p(\widehat{x_0^f} | y_0^{i}, c)$

In [167]:
# BASELINE = OLD
def phatXhat0fX0k_baseline(xhat0f, x0k, c, m = 50, my_j=None):
#     y0ks = tuple(sample_pY0kX0k(x0k) for each in range(n))
#     terms = (pXhat0fY0k(xhat0f, y0k) for y0k in y0ks)

#     foo = partial(pXhat0fY0i, xhat0f)
#     def foo(y0i):
#         return pXhat0fY0i(xhat0f, y0i)
#     terms = (foo(y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=8, prefer="threads")(delayed(foo)(y0i) for y0i in y0is)
#     terms = Parallel(n_jobs=8, prefer="threads")(delayed(foo)(y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=5, verbose=50, batch_size=10, prefer="threads")(delayed(pXhat0fY0i)(xhat0f, y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=-1, verbose=50, batch_size=round(n/12), prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i)(xhat0f, y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))

#     terms = Parallel(n_jobs=-1, batch_size=round(n/12), prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0k_old)(xhat0f, y0k) for y0k in (sample_pY0kX0k(x0k) for each in range(n)))
#     terms = Parallel(n_jobs=-1, batch_size='auto', prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i_old)(xhat0f, y0i) for y0i in (sample_pY0iX0k(x0k) for each in range(m)))
    if my_j is None:
        my_j = -1
    terms = Parallel(n_jobs=my_j, batch_size='auto', prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i_old)(xhat0f, y0i, c) for y0i in (sample_pY0iX0k(x0k) for each in range(m)))
#     terms = (pXhat0fY0k_old(xhat0f, y0k) for y0k in (sample_pY0kX0k(x0k) for each in range(n)))
    s = sum(terms)
    sbar = (1.0 * s) / m
    return sbar

In [168]:
random_context = choice(Cs_t); random_context

"to different people's"

In [169]:
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 200)

0.0019045632398041127

In [170]:
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 50)

0.0018994033198850764

In [171]:
{w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}
[f"p(W' = {w}) → {pW[w]}"
 for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}),
                 key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

[f"p(W' = {w}|r = '⋊.k.ɑ.n.t') → {phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1)}"
 for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}),
                 key=lambda w:phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1), reverse=True)]

{'⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉',
 '⋊.k.ɑ.n.t.n.ɪ.n.t.⋉',
 '⋊.k.ɑ.n.t.æ.k.t.⋉',
 '⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉',
 '⋊.k.ɑ.n.t.ɛ.k.s.t.⋉',
 '⋊.k.ɑ.n.t.ɛ.s.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉'}

["p(W' = ⋊.k.ɑ.n.t.æ.k.t.⋉) → 1.9927084903616688e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉) → 1.1031811727794093e-07",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.s.t.⋉) → 1.7458090784748148e-07",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.k.s.t.⋉) → 3.301887226828777e-07",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉) → 9.314244854919203e-07",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉) → 6.6987445094223545e-09",
 "p(W' = ⋊.k.ɑ.n.t.n.ɪ.n.t.⋉) → 4.2201127768971105e-09",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉) → 3.6577460887984323e-09",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉) → 3.1576348202842305e-08",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉) → 2.946759813831917e-08",
 "p(W' = ⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉) → 8.876113708312564e-10",
 "p(W' = ⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉) → 5.546937969092835e-10",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉) → 2.5203674880187e-10"]

["p(W' = ⋊.k.ɑ.n.t.æ.k.t.⋉|r = '⋊.k.ɑ.n.t') → 0.00196100841521003",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉|r = '⋊.k.ɑ.n.t') → 0.0012095198271750006",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.k.s.t.⋉|r = '⋊.k.ɑ.n.t') → 0.00042056865314207023",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.s.t.⋉|r = '⋊.k.ɑ.n.t') → 0.00043874919895792506",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉|r = '⋊.k.ɑ.n.t') → 0.00029866591762538056",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉|r = '⋊.k.ɑ.n.t') → 0.0001470854939451608",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉|r = '⋊.k.ɑ.n.t') → 7.343700432276226e-05",
 "p(W' = ⋊.k.ɑ.n.t.n.ɪ.n.t.⋉|r = '⋊.k.ɑ.n.t') → 7.729997382290322e-05",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉|r = '⋊.k.ɑ.n.t') → 5.4203076667280166e-05",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉|r = '⋊.k.ɑ.n.t') → 2.2147522541223355e-05",
 "p(W' = ⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉|r = '⋊.k.ɑ.n.t') → 2.2049760976799344e-05",
 "p(W' = ⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉|r = '⋊.k.ɑ.n.t') → 1.1112876507513368e-05",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉|r = '⋊.k.ɑ.n.t') → 5.28724429857172e-06"]

In [172]:
# {w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}

[f"p(W' = {w}) → {pW_C_lookup(w=w, c=random_context)}"
 for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}),
                 key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

[f"p(W' = {w}|r = '⋊.k.ɑ.n.t') → {phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1)}"
 for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}),
                 key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

["p(W' = ⋊.k.ɑ.n.f.ɪ.d.ɪ.n.s.⋉) → 0.0013489271011869973",
 "p(W' = ⋊.k.ɑ.n.v.ɚ.s.eɪ.ʃ.ɪ.n.⋉) → 1.4135549520601253e-05",
 "p(W' = ⋊.k.ɑ.n.t.æ.k.t.⋉) → 9.320544712607438e-06",
 "p(W' = ⋊.k.ɑ.n.f.l.ɪ.k.t.⋉) → 5.288703930927082e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉) → 5.0676920190206445e-06",
 "p(W' = ⋊.k.ɑ.n.s.ɛ.p.t.⋉) → 4.8495897904792925e-06",
 "p(W' = ⋊.k.ɑ.n.tʃ.ɪ.s.⋉) → 2.6595743689763263e-06",
 "p(W' = ⋊.k.ɑ.n.s.ɚ.t.⋉) → 2.136425961306497e-06",
 "p(W' = ⋊.k.ɑ.n.f.ɚ.ə.n.s.⋉) → 1.8078572647336528e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.s.t.⋉) → 1.8078572647336528e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.k.s.t.⋉) → 1.6500248804840168e-06",
 "p(W' = ⋊.k.ɑ.n.s.ɪ.n.t.ɹ.eɪ.t.⋉) → 1.3480448964260746e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉) → 1.275532271469346e-06",
 "p(W' = ⋊.k.ɑ.n.d.ʌ.k.t.⋉) → 1.204263805752367e-06",
 "p(W' = ⋊.k.ɑ.n.k.ɹ.i.t.⋉) → 9.982097494375289e-07",
 "p(W' = ⋊.k.ɑ.n.s.ɪ.n.t.ɹ.eɪ.ʃ.ɪ.n.⋉) → 8.676242733843796e-07",
 "p(W' = ⋊.k.ɑ.n.⋉) → 7.040267015505823e-07",
 "p(W' = ⋊.k.ɑ.n.s.oʊ.l.⋉) → 5.67232738424

["p(W' = ⋊.k.ɑ.n.f.ɪ.d.ɪ.n.s.⋉|r = '⋊.k.ɑ.n.t') → 0.35999279324587546",
 "p(W' = ⋊.k.ɑ.n.v.ɚ.s.eɪ.ʃ.ɪ.n.⋉|r = '⋊.k.ɑ.n.t') → 0.0029769537255634275",
 "p(W' = ⋊.k.ɑ.n.t.æ.k.t.⋉|r = '⋊.k.ɑ.n.t') → 0.0018706158211694163",
 "p(W' = ⋊.k.ɑ.n.f.l.ɪ.k.t.⋉|r = '⋊.k.ɑ.n.t') → 0.0013791913901291658",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉|r = '⋊.k.ɑ.n.t') → 0.0011943326983230845",
 "p(W' = ⋊.k.ɑ.n.s.ɛ.p.t.⋉|r = '⋊.k.ɑ.n.t') → 0.0010565748111712157",
 "p(W' = ⋊.k.ɑ.n.tʃ.ɪ.s.⋉|r = '⋊.k.ɑ.n.t') → 0.0005851783017679133",
 "p(W' = ⋊.k.ɑ.n.s.ɚ.t.⋉|r = '⋊.k.ɑ.n.t') → 0.0004966858526049557",
 "p(W' = ⋊.k.ɑ.n.f.ɚ.ə.n.s.⋉|r = '⋊.k.ɑ.n.t') → 0.0004809414912611388",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.s.t.⋉|r = '⋊.k.ɑ.n.t') → 0.00043281177365148664",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.k.s.t.⋉|r = '⋊.k.ɑ.n.t') → 0.00041265195758632874",
 "p(W' = ⋊.k.ɑ.n.s.ɪ.n.t.ɹ.eɪ.t.⋉|r = '⋊.k.ɑ.n.t') → 0.00031350765627728407",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉|r = '⋊.k.ɑ.n.t') → 0.00022836045389359409",
 "p(W' = ⋊.k.ɑ.n.d.ʌ.k.t.⋉|r = '⋊.k.ɑ.n.t') → 0.00025600748

In [173]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')

2773

In [174]:
'⋊.k.ɑ.n.t'
len( ds2t('⋊.k.ɑ.n.t') )
len( ds2t('⋊.k.ɑ.n.t') ) - 1
retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉', 4)

np.allclose( retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉', 4), 
             pY0k_X0ks[4][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')])

'⋊.k.ɑ.n.t'

5

4

array([[ 0.04390334,  0.01805178],
       [ 0.00700978,  0.01374583],
       [ 0.02545656,  0.00721312],
       [ 0.00700978,  0.00711758],
       [ 0.00682531,  0.00692774],
       [ 0.00700978,  0.00721312],
       [ 0.00682531,  0.0070233 ],
       [ 0.00700978,  0.00721312],
       [ 0.00682531,  0.00711758],
       [ 0.00700978,  0.00721312],
       [ 0.0051651 ,  0.00619171],
       [ 0.65246265,  0.00711758],
       [ 0.02545656,  0.00721312],
       [ 0.00700978,  0.00711758],
       [ 0.00700978,  0.00721312],
       [ 0.00700978,  0.01374583],
       [ 0.00700978,  0.00711758],
       [ 0.00682531,  0.00711758],
       [ 0.00700978,  0.00721312],
       [ 0.00700978,  0.00721312],
       [ 0.00700978,  0.00721312],
       [ 0.00682531,  0.00711758],
       [ 0.00664084,  0.00702074],
       [ 0.00682531,  0.0070233 ],
       [ 0.00700978,  0.01805178],
       [ 0.00682531,  0.00692774],
       [ 0.00700978,  0.00619171],
       [ 0.00700978,  0.59565208],
       [ 0.00700978,

True

# Definition of core calculation

In [175]:
from random import choice

In [176]:
random_wordform = choice(Ws_t)
random_wordform_length = len(ds2t(random_wordform))
random_wordform_idx = Ws_t.index(random_wordform)
random_wordform, random_wordform_idx, random_wordform_length

('⋊.d.eɪ.n.dʒ.ɚ.⋉', 606, 7)

In [177]:
type(pW_C)
pW_C.shape
pW_C[:,0].shape

numpy.ndarray

(9172, 106295)

(9172,)

In [178]:
Cs_t[Cs_t.index('a couple of')]

'a couple of'

In [179]:
random_context = 'a couple of'

In [180]:
# pX0f = pW_C[:,Cs_t.index('a couple of')]
# pX0f_torch = torch.from_numpy(pX0f)
pW_C_torch = torch.from_numpy(pW_C)

In [181]:
def get_pW_t(c_idx):
    return pW_C_torch[:,c_idx]

In [182]:
# if r:
    CMsByLengthByWordformIndex = pY0k_X0ks
    CMsByLengthByWordformIndex_torch = [torch.from_numpy(each) for each in CMsByLengthByWordformIndex]
    exactCMsByLengthByWordformIndex = pY0f_X0fs
    exatCMsByLengthByWordformIndex_torch = [torch.from_numpy(each) for each in exactCMsByLengthByWordformIndex]
# else:
#     xCMsByLengthByWordformIndex_torch
#     raise Exception('Feature not implemented yet')
#     CMsByLengthByWordformIndex = pY0f_X0ks
#     CMsByLengthByWordformIndex_torch = [torch.from_numpy(each) for each in CMsByLengthByWordformIndex]
#     pass

In [183]:
random_wordform
ds2t(random_wordform)[1:-1]

'⋊.d.eɪ.n.dʒ.ɚ.⋉'

('d', 'eɪ', 'n', 'dʒ', 'ɚ')

In [184]:
random_wordform
ds2t(random_wordform)
random_wordform_idx
random_wordform_length
random_wordform_CM = CMsByLengthByWordformIndex_torch[random_wordform_length][random_wordform_idx]
'--'
retrieve_CM_for(random_wordform, len(ds2t(random_wordform)))
np.allclose(retrieve_CM_for(random_wordform, len(ds2t(random_wordform))),
            CMsByLengthByWordformIndex[random_wordform_length][random_wordform_idx])
'--'
CMsByLengthByWordformIndex_torch[random_wordform_length][random_wordform_idx].shape
random_wordform_CM

'⋊.d.eɪ.n.dʒ.ɚ.⋉'

('⋊', 'd', 'eɪ', 'n', 'dʒ', 'ɚ', '⋉')

606

7

'--'

array([[ 0.00700978,  0.01358527,  0.00961539,  0.01167415,  0.00700978],
       [ 0.00700978,  0.00712887,  0.00961539,  0.01167415,  0.00700978],
       [ 0.00700978,  0.00712887,  0.00961539,  0.01167415,  0.00700978],
       [ 0.32060505,  0.01353596,  0.00948803,  0.05409592,  0.00682531],
       [ 0.00682531,  0.00684682,  0.00923497,  0.39493876,  0.02508762],
       [ 0.00700978,  0.71057347,  0.00961539,  0.01167415,  0.00700978],
       [ 0.02527209,  0.00694126,  0.00936236,  0.01136694,  0.00682531],
       [ 0.00700978,  0.00712887,  0.00961539,  0.04783478,  0.00700978],
       [ 0.00682531,  0.00703444,  0.00948803,  0.01151952,  0.00700978],
       [ 0.00700978,  0.00712887,  0.00961539,  0.01167415,  0.00700978],
       [ 0.0051651 ,  0.00611938,  0.00825381,  0.01002103,  0.00700978],
       [ 0.00682531,  0.00703444,  0.00948803,  0.01151952,  0.00700978],
       [ 0.00700978,  0.00712887,  0.00961539,  0.01167415,  0.00700978],
       [ 0.00700978,  0.00703444,  0.0

True

'--'

torch.Size([38, 5])

tensor([[0.0070, 0.0136, 0.0096, 0.0117, 0.0070],
        [0.0070, 0.0071, 0.0096, 0.0117, 0.0070],
        [0.0070, 0.0071, 0.0096, 0.0117, 0.0070],
        [0.3206, 0.0135, 0.0095, 0.0541, 0.0068],
        [0.0068, 0.0068, 0.0092, 0.3949, 0.0251],
        [0.0070, 0.7106, 0.0096, 0.0117, 0.0070],
        [0.0253, 0.0069, 0.0094, 0.0114, 0.0068],
        [0.0070, 0.0071, 0.0096, 0.0478, 0.0070],
        [0.0068, 0.0070, 0.0095, 0.0115, 0.0070],
        [0.0070, 0.0071, 0.0096, 0.0117, 0.0070],
        [0.0052, 0.0061, 0.0083, 0.0100, 0.0070],
        [0.0068, 0.0070, 0.0095, 0.0115, 0.0070],
        [0.0070, 0.0071, 0.0096, 0.0117, 0.0070],
        [0.0070, 0.0070, 0.0095, 0.0115, 0.0068],
        [0.0070, 0.0071, 0.6021, 0.0117, 0.0070],
        [0.0070, 0.0071, 0.0096, 0.0117, 0.0070],
        [0.0070, 0.0070, 0.0095, 0.0115, 0.0068],
        [0.0068, 0.0070, 0.0095, 0.0115, 0.0070],
        [0.3206, 0.0071, 0.0096, 0.0222, 0.0070],
        [0.0070, 0.0071, 0.0096, 0.0812, 0.0070],


## Channel sampler

In [185]:
# if not r:
    def depthSampler2a_t(CM, m=1):
        '''
        Given 
         - a sequence of channel distributions (i.e. a pytorch tensor)
           associated with some prefix x_0^i of the lexicon
         - a number of samples m
        this returns m samples from the channel distribution p(Y_0^i|x_0^i),
        where each 'sample' (i.e. channel string) is a sequence of one hot vectors.
        
        Let l = i+1:
        |depthSampler2a_t(|CM| = (|Y1|, l), m)| = (m, l, |Y1|)
        '''
        stack = torch.zeros((m, CM.shape[1], CM.shape[0]))
#         print(f'{stack.dtype}')
        for eachStack in torch.arange(m):
            for i in torch.arange(CM.shape[1]):
                stack[eachStack, i] = torch.distributions.Multinomial(1, CM[:,i]).sample()
        return stack
#         return torch.squeeze(stack)
# else:
#     def depthSampler2a_t(xCM, m=1):
#         stack = torch.zeros((m, xCM.shape[1], xCM.shape[0]))
#         for eachStack in torch.arange(m):
#             for i in torch.arange(xCM.shape[1]):
#                 stack[eachStack, i] = torch.distributions.Multinomial(1, xCM[:,i]).sample()
#         return stack

In [ ]:
#FIXME copy what you need from the segmental sequence notebook to interpret these samples

In [186]:
random_wordform_CM.shape

torch.Size([38, 5])

In [187]:
random_wordform
sampled_channel_sequences = depthSampler2a_t(random_wordform_CM, m=2)
sampled_channel_sequences
channelSeqOHs2ds(sampled_channel_sequences.numpy()[0], False)
channelSeqOHs2ds(sampled_channel_sequences.numpy()[0], True)
sampled_channel_sequences.shape

'⋊.d.eɪ.n.dʒ.ɚ.⋉'

tensor([[[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
     

'd.eɪ.n.dʒ.ɚ'

'⋊.d.eɪ.n.dʒ.ɚ'

torch.Size([2, 5, 38])

In [188]:
random_wordform
sampled_channel_sequences2 = depthSampler2a_t(random_wordform_CM, m=100)
[channelSeqOHs2ds(each.numpy(), True) for each in sampled_channel_sequences2]

'⋊.d.eɪ.n.dʒ.ɚ.⋉'

['⋊.t.dʒ.n.ʒ.ɚ',
 '⋊.t.eɪ.n.ɑ.ɚ',
 '⋊.t.eɪ.n.tʃ.k',
 '⋊.d.eɪ.n.eɪ.ɚ',
 '⋊.ʃ.eɪ.n.ʃ.ɚ',
 '⋊.d.eɪ.n.tʃ.ɚ',
 '⋊.d.d.aʊ.dʒ.ɚ',
 '⋊.eɪ.eɪ.u.dʒ.t',
 '⋊.t.eɪ.i.dʒ.ɚ',
 '⋊.aʊ.oʊ.n.ʊ.ɚ',
 '⋊.d.eɪ.n.ʃ.ɚ',
 '⋊.d.eɪ.n.dʒ.ɚ',
 '⋊.w.eɪ.ɹ.dʒ.ɚ',
 '⋊.ʃ.eɪ.n.ŋ.ɹ',
 '⋊.t.eɪ.n.tʃ.ɚ',
 '⋊.t.eɪ.n.ʒ.ɚ',
 '⋊.d.eɪ.n.dʒ.d',
 '⋊.d.eɪ.n.ʌ.ɚ',
 '⋊.d.eɪ.n.aʊ.ɚ',
 '⋊.tʃ.eɪ.n.dʒ.ɚ',
 '⋊.t.æ.n.tʃ.ɚ',
 '⋊.d.l.n.dʒ.ɚ',
 '⋊.d.eɪ.ŋ.ʒ.ɚ',
 '⋊.ɹ.æ.n.dʒ.ɚ',
 '⋊.d.eɪ.n.j.ɹ',
 '⋊.m.eɪ.n.ɹ.ɚ',
 '⋊.d.ʌ.ɑ.w.ɚ',
 '⋊.t.eɪ.ð.dʒ.θ',
 '⋊.d.ɹ.n.ʌ.ɚ',
 '⋊.t.eɪ.n.d.ɚ',
 '⋊.t.eɪ.n.ʌ.ɚ',
 '⋊.ɹ.eɪ.ɚ.n.ɚ',
 '⋊.d.eɪ.ɑ.eɪ.ɚ',
 '⋊.ɔɪ.eɪ.n.i.ɚ',
 '⋊.d.z.n.dʒ.ɚ',
 '⋊.ɹ.eɪ.æ.u.ɚ',
 '⋊.ɹ.eɪ.ɛ.dʒ.ɚ',
 '⋊.t.eɪ.ʌ.w.ɚ',
 '⋊.ð.eɪ.eɪ.dʒ.ɚ',
 '⋊.d.eɪ.n.f.ɚ',
 '⋊.d.eɪ.n.dʒ.ɚ',
 '⋊.tʃ.eɪ.n.ʌ.ɚ',
 '⋊.ɛ.eɪ.d.d.ɚ',
 '⋊.t.eɪ.θ.dʒ.ɚ',
 '⋊.d.s.d.tʃ.ɚ',
 '⋊.d.eɪ.n.dʒ.ɚ',
 '⋊.t.eɪ.m.tʃ.ɚ',
 '⋊.eɪ.æ.n.dʒ.ð',
 '⋊.t.ɹ.n.dʒ.b',
 '⋊.d.ʌ.ð.g.ɚ',
 '⋊.d.tʃ.n.dʒ.ɚ',
 '⋊.d.j.n.dʒ.g',
 '⋊.t.p.dʒ.ŋ.h',
 '⋊.aɪ.eɪ.ʊ.ð.ɚ',
 '⋊.w.v.n.dʒ.ɚ',
 '⋊.f.eɪ.n.dʒ.ɚ',
 '⋊.t.eɪ.n.dʒ.ɚ',
 '⋊

In [189]:
random_wordform
sampled_channel_sequences3 = [sample_pY0iX0k(random_wordform) for each in range(50)]; sampled_channel_sequences3

'⋊.d.eɪ.n.dʒ.ɚ.⋉'

['⋊.d.eɪ.tʃ.dʒ.ɚ',
 '⋊.f.æ.ɪ.g.w',
 '⋊.f.eɪ.j.dʒ.ɚ',
 '⋊.t.oʊ.n.ð.g',
 '⋊.t.eɪ.oʊ.dʒ.ɚ',
 '⋊.t.h.ɪ.dʒ.ɹ',
 '⋊.t.eɪ.n.θ.ɚ',
 '⋊.t.eɪ.n.j.ɚ',
 '⋊.ɹ.eɪ.n.t.ɚ',
 '⋊.ɚ.ɪ.n.b.ɚ',
 '⋊.d.eɪ.s.aɪ.p',
 '⋊.d.eɪ.n.l.ɚ',
 '⋊.ɹ.ŋ.n.dʒ.n',
 '⋊.f.eɪ.ɪ.dʒ.ɚ',
 '⋊.ʃ.eɪ.aʊ.dʒ.j',
 '⋊.f.eɪ.n.dʒ.ɚ',
 '⋊.d.eɪ.n.ʌ.ɚ',
 '⋊.t.eɪ.ʒ.dʒ.g',
 '⋊.tʃ.eɪ.n.h.ŋ',
 '⋊.l.eɪ.n.dʒ.ɚ',
 '⋊.ɛ.eɪ.n.aʊ.ɚ',
 '⋊.z.ɹ.n.m.ɚ',
 '⋊.t.eɪ.eɪ.dʒ.aɪ',
 '⋊.d.ɔɪ.f.dʒ.ɚ',
 '⋊.t.eɪ.n.g.ɚ',
 '⋊.d.eɪ.n.ɑ.ɚ',
 '⋊.t.eɪ.n.d.ɚ',
 '⋊.d.ŋ.ɛ.ɑ.ɚ',
 '⋊.t.eɪ.æ.tʃ.ɚ',
 '⋊.v.eɪ.n.dʒ.ɚ',
 '⋊.t.eɪ.n.tʃ.p',
 '⋊.ɹ.eɪ.ʌ.tʃ.ɚ',
 '⋊.d.eɪ.n.eɪ.ɚ',
 '⋊.g.eɪ.n.ʌ.ɚ',
 '⋊.t.eɪ.ɚ.p.ɚ',
 '⋊.t.d.aʊ.dʒ.ɚ',
 '⋊.θ.eɪ.n.ʒ.ɚ',
 '⋊.t.eɪ.t.eɪ.ɚ',
 '⋊.f.eɪ.n.tʃ.ɚ',
 '⋊.ɹ.eɪ.aɪ.d.ɚ',
 '⋊.t.eɪ.ʌ.h.n',
 '⋊.ɹ.k.n.ʒ.ɚ',
 '⋊.ɹ.eɪ.n.dʒ.ʊ',
 '⋊.ʒ.eɪ.t.i.w',
 '⋊.t.ɪ.n.ʌ.ð',
 '⋊.u.eɪ.n.l.n',
 '⋊.t.eɪ.n.m.ʃ',
 '⋊.d.eɪ.p.ð.ɚ',
 '⋊.d.eɪ.ʌ.dʒ.j',
 '⋊.l.eɪ.n.dʒ.ɚ']

In [190]:
len(ds2t('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h'))
y0kOHmap('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h')
y0kOHmap('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h').shape
np.array(list(map(y0kOHmap, sampled_channel_sequences3))).shape
np.array(list(map(y0kOHmap, [sample_pY0iX0k(random_wordform) for each in range(50)])))

7

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,

(6, 38)

(50, 5, 38)

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0., 

In [191]:
torch.tensor(sample_pY0iX0k(random_wordform, debug=True)[1]) == random_wordform_CM

tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=torch.uint8)

In [ ]:
# channelSeqOHs2ds(np.array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#           0., 0., 0., 0.]]))

## Sample-parallel posterior estimator

In [215]:
# if not r:
#     def pXhat0fX0i_pxt(xhat0f_idx, x0k_CM=None, m = 50, x0k=None):
    def pXhat0fX0i_pxt(xhat0f_idx, x0k_CM, c_idx, m = 50):
        '''
        If ⋊ x_0 x_1 ... x_i x_k denotes
        the produced prefix r of the speaker's intended wordform w,
         where
           x_i = the last fully produced segment
           k = i+1
           x_k = the next segment
           |⋊ x_0 x_1 ... x_i x_k| = l + 2
           l = i+1 = the length of the produced prefix (not incl. 
                     the left word edge or the next segment)
        
        then
        
        x0k_CM :: (|Y1|, l) is the associated l-length stack of size |Y1| vectors
         where x0k_CM[:,j] is the distribution p(Y_j|X0^k has been produced)
           
        xhat0f_idx is the index of a wordform w' s.t. 
        
        pXhat0fX0i_pxt(xhat0f_idx, x0k_CM) ≈ p(w'|r)
        '''
        shape_info = False
        pW = get_pW_t(c_idx)
#         if x0k_CM is None and x0k is None:
#             raise Exception('Must specify one of x0k_CM or x0k.')
        
        # Computation proceeds in two steps:
        # calculate the denominator p(y0i) for m = 50 sampled y0i's
        # calculate the numerator p(y0i|w')p(w') for m = 50 sampled y0i's
        
        
    #     xhat0f_idx = Wmap[xhat0f]

    #     l = len(ds2t(x0i))
    #     x0k_CM = CMsByPrefixIndex[prefixMap[x0i]]

    #     my_Q_l = CMsByLengthByWordformIndex[l - 2]

        #Collect m samples from p(Y0i|X0k) = p(Y0i|r), 
        # where each sample is an l-length stack of one-hot vectors
        # where each one-hot vector corresponds to a channel symbol:
        Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
#         if x0k_CM is not None:
#             Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
#             if x0k is not None:
# #                 print('foo')
#                 print(type(x0k_CM))
#                 print(x0k_CM.shape)
#                 generated_CM = sample_pY0iX0k(x0k, debug=True)[1]
#                 print(type(generated_CM))
#                 print(generated_CM.shape)
#                 generated_CM_t = torch.tensor(generated_CM)
#                 assert torch.allclose(x0k_CM, torch.tensor(sample_pY0iX0k(x0k, debug=True)[1]))
# #                 print('bar')
#         else:
#             Y_prime = torch.tensor(list(map(y0kOHmap, [sample_pY0iX0k(x0k) for each in range(m)])))
#         print('baz')
        l = Y_prime.shape[1]# + 1
        if shape_info:
            print(f'|Y_prime| = {Y_prime.shape} = (m, l, |Y1|)')

        #Grab ALL the relevant-prefix-length channel matrices 
        # for every segmental wordform in the lexicon:
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 3]#.float()
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 1]#.float() #:: (|W|, |Y1|, l)
        my_Q_l = CMsByLengthByWordformIndex_torch[l+2]#.float() #:: (|W|, |Y1|, l)
        if shape_info:
            print(f'|Q_l| = {my_Q_l.shape} = (|W|=n, |Y1|, l)')
    
#         print(f'{Y_prime.dtype}, {my_Q_l.dtype}')
    
        # NORMALIZATION
        V_prime = torch.einsum('mli,kil->mkl', Y_prime, my_Q_l)  # :: (m,n,l)
#         print(f"|V'| = {V_prime.shape} = (m, n, l)")
        M_prime = torch.prod(V_prime, 2) # :: (m,n)
#         print(f"|M'| = {M_prime.shape} = (m, n)")
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, 1) <- prior probabilities of each of the m sampled channel prefixes
        N_prime = torch.matmul(M_prime, pW)
#         print(f"|N'| = {N_prime.shape} = (m, )")
        Z_prime = 1.0 / N_prime # :: (m, 1)
#         print(f"|Z'| = {Z_prime.shape} = (m, )")
        if shape_info:
            print(f"|V'| = {V_prime.shape} = (m, n, l)")
            print(f"|M'| = {M_prime.shape} = (m, n)")
            print(f"|N'| = {N_prime.shape} = (m, )")
            print(f"|Z'| = {Z_prime.shape} = (m, )")

        # NUMERATOR
        L_w = my_Q_l[xhat0f_idx]#.float()
        V_prime_w = torch.einsum('mij,ji->mi',Y_prime, L_w)
#         print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
        O_w = torch.prod(V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
#         print(f"|O_w| = {O_w.shape} = (m, )")
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes
        U_w = pW[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes
#         print(f"|U_w| = {U_w.shape} = (m, )")
        if shape_info:
            print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
            print(f"|O_w| = {O_w.shape} = (m, )")
            print(f"|U_w| = {U_w.shape} = (m, )")
        
        
        if shape_info:
            E = torch.dot(Z_prime, U_w) / m
            print(f"|E| = scalar: type(E) = {type(E)}")
        return torch.dot(Z_prime, U_w) / m

    #     return torch.dot( 1.0 / torch.matmul(torch.prod(torch.einsum('mli,kil->mkl', Y_prime, my_Q_l), 2), pX0f_torch) , 
    #        pX0f_torch[xhat0f_idx] * torch.prod(torch.einsum('mij,ji->mi',Y_prime, L_w), 1) ) / m
# else:
#     def pXhat0fX0k_pxt(xhat0f_idx, x0i_xCM, m = 50):
#     #     xhat0f_idx = Wmap[xhat0f]

#     #     l = len(ds2t(x0i))
#     #     x0i_xCM = xCMsByPrefixIndex[prefixMap[x0i]]

#     #     my_Q_l = xCMsByLengthByWordformIndex[l - 2]

#         Y_prime = depthSampler2a_t(x0i_xCM, m)#.float()
#         l = Y_prime.shape[1] + 1

#         my_Q_l = xCMsByLengthByWordformIndex_torch[l - 3]#.float()

#         # NORMALIZATION
#         V_prime = torch.einsum('mli,kil->mkl', Y_prime, my_Q_l)  # :: (m,n,l)
#         M_prime = torch.prod(V_prime, 2) # :: (m,n)
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, 1) <- prior probabilities of each of the m sampled channel prefixes
#         Z_prime = 1.0 / N_prime # :: (m, 1)

#         # NUMERATOR
#         L_w = my_Q_l[xhat0f_idx]#.float()
#         V_prime_w = torch.einsum('mij,ji->mi',Y_prime, L_w)
#         O_w = torch.prod(V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes

#         return torch.dot(Z_prime, U_w) / m

#     #     return torch.dot( 1.0 / torch.matmul(torch.prod(torch.einsum('mli,kil->mkl', Y_prime, my_Q_l), 2), pX0f_torch) , 
#     #        pX0f_torch[xhat0f_idx] * torch.prod(torch.einsum('mij,ji->mi',Y_prime, L_w), 1) ) / m

In [210]:
def pXhat0fX0i_pxtn(xhat0f_idx, x0k_CM, c_idx, m = 50):
    return pXhat0fX0i_pxt(xhat0f_idx, x0k_CM, c_idx, m = 50).numpy()

In [201]:
def prefixTox0kCM(r):
    source_wf = list(wordsWithPrefix(r, Ws))[0]
    source_wf_idx = Ws_t.index(source_wf)
    
    r_t = ds2t(r)
    total_x0k_length = len(r_t)# includes left edge symbol + upcoming segment
#     prefix_length_noWE = total_x0k_length - 1 #includes upcoming segment
#     prefix_length_produced_so_far = prefix_length_noWE - 1
#     offset = 1
#     x0k_cm = CMsByLengthByWordformIndex_torch[prefix_length_produced_so_far - offset][source_wf_idx]
    x0k_cm = CMsByLengthByWordformIndex_torch[total_x0k_length][source_wf_idx]
    return x0k_cm

In [217]:
CMsByLengthByWordformIndex_torch[3].shape
CMsByLengthByWordformIndex_torch[4].shape
len(CMsByLengthByWordformIndex_torch)

torch.Size([9172, 38, 1])

torch.Size([9172, 38, 2])

19

In [203]:
len(Ws_t)

9172

In [219]:
def wordformTox0kCM(w=None, w_idx=None):
#     if w is None and w_idx is None:
#         raise Exception('At least one argument must be specified.')
    if w is None:
        w = Ws_t[w_idx]
        w_t = ds2t(w)
    else:
        w_t = ds2t(w)
        w_idx = Ws_t.index(w)
    
    total_x0f_length = len(w_t) #includes both edge symbols
#     word_length_noLE = total_x0f_length - 1 #still includes right edge symbol = "upcoming segment"
#     word_length_noRE = word_length_noLE - 1
#     offset = 2
#     assert word_length_produced_so_far - offset >= 0 , f"{word_length_produced_so_far} - {offset} < 0; w = {w}"
#     assert word_length_produced_so_far - offset < len(CMsByLengthByWordformIndex_torch), f"{word_length_produced_so_far} - {offset} >= {len(CMsByLengthByWordformIndex_torch)}; w = {w}"
#     assert w_idx in range(CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset].shape[0]), f"{w_idx}; w = {w}"
#     x0f_cm = CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset][w_idx]
    x0f_cm = CMsByLengthByWordformIndex_torch[total_x0f_length][w_idx]
    return x0f_cm

# def wordformTox0fCM(w=None, w_idx=None):
# #     if w is None and w_idx is None:
# #         raise Exception('At least one argument must be specified.')
#     if w is None:
#         w = Ws_t[w_idx]
#         w_t = ds2t(w)
#     else:
#         w_t = ds2t(w)
#         w_idx = Ws_t.index(w)
    
#     total_x0f_length = len(w_t) #includes both edge symbols
# #     word_length_noLE = total_x0f_length - 1 #still includes right edge symbol = "upcoming segment"
# #     word_length_noRE = word_length_noLE - 1
# #     offset = 2
# #     assert word_length_produced_so_far - offset >= 0 , f"{word_length_produced_so_far} - {offset} < 0; w = {w}"
# #     assert word_length_produced_so_far - offset < len(CMsByLengthByWordformIndex_torch), f"{word_length_produced_so_far} - {offset} >= {len(CMsByLengthByWordformIndex_torch)}; w = {w}"
# #     assert w_idx in range(CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset].shape[0]), f"{w_idx}; w = {w}"
# #     x0f_cm = CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset][w_idx]
#     x0f_cm = exactCMsByLengthByWordformIndex_torch[total_x0f_length][w_idx]
#     return x0f_cm

In [204]:
len(ds2t('⋊.m.ɪ.n.ɪ.s.t'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t').shape

len(ds2t('⋊.m.ɪ.n.ɪ.s.t.ɚ'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t.ɚ'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t.ɚ').shape

len(ds2t('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉').shape

7

5

torch.Size([38, 5])

8

6

torch.Size([38, 6])

9

7

torch.Size([38, 7])

In [206]:
random_source_prefix
len(ds2t(random_source_prefix)) - 2
prefixTox0kCM(random_source_prefix).shape
prefixTox0kCM(random_source_prefix)

'⋊.æ.k.t.ɪ.v.ə.t'

6

torch.Size([38, 6])

tensor([[0.1730, 0.0093, 0.0090, 0.0083, 0.0090, 0.0626],
        [0.0070, 0.0093, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0439, 0.0093, 0.0090, 0.0158, 0.0090, 0.0133],
        [0.0070, 0.0092, 0.0089, 0.0082, 0.0089, 0.0132],
        [0.0068, 0.0090, 0.0087, 0.0079, 0.0086, 0.0128],
        [0.0255, 0.0351, 0.0090, 0.0389, 0.0090, 0.1367],
        [0.0068, 0.0091, 0.0088, 0.0081, 0.0087, 0.0130],
        [0.0070, 0.0178, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0068, 0.0176, 0.0089, 0.0082, 0.0089, 0.0132],
        [0.0070, 0.0093, 0.0090, 0.0207, 0.0090, 0.0133],
        [0.0052, 0.0080, 0.0077, 0.0071, 0.0077, 0.0114],
        [0.0068, 0.5897, 0.0170, 0.0082, 0.0089, 0.0132],
        [0.0070, 0.0093, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0070, 0.0092, 0.0089, 0.0082, 0.0089, 0.0132],
        [0.0070, 0.0093, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0070, 0.0093, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0070, 0.0092, 0.0089, 0.0082, 0.0089, 0.0132],
        [0.006

In [211]:
# for convenience
def pXhat0fX0i_pxtn_conv(w, r, c, m = 50):
    w_idx = Ws_t.index(w)

#     r_idx = Ps_t.index(r)
    source_wf = list(wordsWithPrefix(r, Ws))[0]
    source_wf_idx = Ws_t.index(source_wf)
    
    total_x0k_length = len(ds2t(r))# includes left edge symbol + upcoming segment
#     prefix_length_noWE = total_x0k_length - 1 #includes upcoming segment
#     prefix_length_produced_so_far = prefix_length_noWE - 1
#     offset = 1
#     x0k_cm = CMsByLengthByWordformIndex_torch[prefix_length_produced_so_far - offset][source_wf_idx]
    x0k_cm = CMsByLengthByWordformIndex_torch[total_x0k_length][source_wf_idx]
    
    c_idx = Cs_t.index(c)
    
    return pXhat0fX0i_pxtn(w_idx, x0k_cm, c_idx, m=m)

In [207]:
random_source_prefix
len(ds2t(random_source_prefix)) - 2
CMsByLengthByWordformIndex_torch[len(ds2t(random_source_prefix))][Ws_t.index(list(wordsWithPrefix(random_source_prefix, Ws))[0])].shape
CMsByLengthByWordformIndex_torch[len(ds2t(random_source_prefix))][Ws_t.index(list(wordsWithPrefix(random_source_prefix, Ws))[0])]

'⋊.æ.k.t.ɪ.v.ə.t'

6

torch.Size([38, 6])

tensor([[0.1730, 0.0093, 0.0090, 0.0083, 0.0090, 0.0626],
        [0.0070, 0.0093, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0439, 0.0093, 0.0090, 0.0158, 0.0090, 0.0133],
        [0.0070, 0.0092, 0.0089, 0.0082, 0.0089, 0.0132],
        [0.0068, 0.0090, 0.0087, 0.0079, 0.0086, 0.0128],
        [0.0255, 0.0351, 0.0090, 0.0389, 0.0090, 0.1367],
        [0.0068, 0.0091, 0.0088, 0.0081, 0.0087, 0.0130],
        [0.0070, 0.0178, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0068, 0.0176, 0.0089, 0.0082, 0.0089, 0.0132],
        [0.0070, 0.0093, 0.0090, 0.0207, 0.0090, 0.0133],
        [0.0052, 0.0080, 0.0077, 0.0071, 0.0077, 0.0114],
        [0.0068, 0.5897, 0.0170, 0.0082, 0.0089, 0.0132],
        [0.0070, 0.0093, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0070, 0.0092, 0.0089, 0.0082, 0.0089, 0.0132],
        [0.0070, 0.0093, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0070, 0.0093, 0.0090, 0.0083, 0.0090, 0.0133],
        [0.0070, 0.0092, 0.0089, 0.0082, 0.0089, 0.0132],
        [0.006

In [214]:
(random_source_wordform, random_source_prefix, random_context)
pXhat0fX0i_pxtn_conv(random_source_wordform, random_source_prefix, random_context)

('⋊.ɹ.ɪ.k.u.p.ɚ.eɪ.t.⋉', '⋊.æ.k.t.ɪ.v.ə.t', 'a couple of')

|Y_prime| = torch.Size([50, 6, 38]) = (m, l, |Y1|)
|Q_l| = torch.Size([9172, 38, 6]) = (|W|=n, |Y1|, l)
|V'| = torch.Size([50, 9172, 6]) = (m, n, l)
|M'| = torch.Size([50, 9172]) = (m, n)
|N'| = torch.Size([50]) = (m, )
|Z'| = torch.Size([50]) = (m, )
|V'_w| = torch.Size([50, 6]) = (m, l)
|O_w| = torch.Size([50]) = (m, )
|U_w| = torch.Size([50]) = (m, )
|E| = scalar: type(E) = <class 'torch.Tensor'>


array(6.984266599803803e-10)

## Testing

In [ ]:
# random_wordform
# random_wordform_length
# retrieve_CM_for(random_wordform, random_wordform_length-2).shape
# np.allclose(retrieve_CM_for(random_wordform, random_wordform_length-2),
#             random_wordform_CM.numpy())
# # random_wordform_CM = CMsByLengthByWordformIndex_torch[random_wordform_length-3][random_wordform_idx]
# random_wordform_idx
# ' '
# Y_PRIME = depthSampler2a_t(random_wordform_CM, m=2); Y_PRIME.shape
# L = Y_PRIME.shape[1]; L
# pXhat0fX0i_pxt(random_wordform_idx, random_wordform_CM)
# # CMsByLengthByWordformIndex_torch[L - 3].shape # <<< this should either be (|Ws| x |Y1s| x ?)

In [ ]:
# len(Ws_t)

In [220]:
# random_context = choice(Cs_t); random_context
random_context = 'a couple of'
random_context_idx = Cs_t.index(random_context); random_context_idx

1753

In [222]:
random_wordform, random_wordform, random_context
pXhat0fX0i_pxt(random_wordform_idx, 
               random_wordform_CM,
               random_context_idx)

('⋊.d.eɪ.n.dʒ.ɚ.⋉', '⋊.d.eɪ.n.dʒ.ɚ.⋉', 'a couple of')

tensor(0.3074)

In [224]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')

'⋊.k.ɑ.n.t'
len( ds2t('⋊.k.ɑ.n.t') )
len( ds2t('⋊.k.ɑ.n.t') ) - 2
retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉', 5)

np.allclose( retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉', 5), 
             CMsByLengthByWordformIndex[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')])

2773

'⋊.k.ɑ.n.t'

5

3

array([[ 0.04390334,  0.01805178,  0.00911065],
       [ 0.00700978,  0.01374583,  0.00911065],
       [ 0.02545656,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00682531,  0.00692774,  0.01700733],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00682531,  0.0070233 ,  0.0088709 ],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00682531,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.0051651 ,  0.00619171,  0.00782054],
       [ 0.65246265,  0.00711758,  0.00898997],
       [ 0.02545656,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.6450247 ],
       [ 0.00700978,  0.01374583,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00682531,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00721312,  0.009

True

In [223]:
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
               CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
               Cs_t.index('a couple of'),
               m = 50)

tensor(0.0042)

In [225]:
# sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 10) for each in range(10)])/10
# sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 50) for each in range(10)])/10
# sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 100) for each in range(10)])/10
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 250)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 500)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 1000)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 2500)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 5000)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 10000)
f"m = 10    → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 10) for each in range(10)])/10}"
f"m = 50    → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 50) for each in range(10)])/10}"
f"m = 100   → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 100) for each in range(10)])/10}"
f"m = 250   → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 250)}"
f"m = 500   → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 500)}"
f"m = 1000  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 1000)}"
f"m = 2500  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 2500)}"
f"m = 5000  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 5000)}"
f"m = 10000 → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', random_context, 10000)}"

'm = 10    → 0.003164642586325636'

'm = 50    → 0.002886675234072019'

'm = 100   → 0.0035419368321361194'

'm = 250   → 0.003334126439270235'

'm = 500   → 0.0031120691806189094'

'm = 1000  → 0.0030770178044458056'

'm = 2500  → 0.003148030417213959'

'm = 5000  → 0.003179217755660128'

'm = 10000 → 0.0031508991794249644'

In [ ]:
#f"m = 50   → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 50)}"
#f"m = 100  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 100)}"
#f"m = 200  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 200)}"
#f"m = 500  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 500)}"
#f"m = 1000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 1000)}"
#f"m = 2000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 2000)}"
#f"m = 5000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 5000)}"

In [226]:
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 50)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 100)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 200)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 500)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 1000)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 2000)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 5000)
f"m = 50   → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')], random_context_idx, m = 50)}"
f"m = 100  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')], random_context_idx, m = 100)}"
f"m = 200  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')], random_context_idx, m = 200)}"
f"m = 500  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')], random_context_idx, m = 500)}"
f"m = 1000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')], random_context_idx, m = 1000)}"
f"m = 2000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')], random_context_idx, m = 2000)}"
f"m = 5000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')], random_context_idx, m = 5000)}"

'm = 50   → 0.0028823458975250054'

'm = 100  → 0.0029037710702363674'

'm = 200  → 0.003329511139180105'

'm = 500  → 0.0027281052993875896'

'm = 1000 → 0.003117716435645003'

'm = 2000 → 0.0031775255119745743'

'm = 5000 → 0.003098023984188958'

In [227]:
%%timeit

pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
              CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
              Cs_t.index('a couple of'), 
              m = 50)

52.4 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [228]:
%%timeit

pXhat0fX0i_pxtn(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
                CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
                Cs_t.index('a couple of'),
                m = 50)

46.9 ms ± 4.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [229]:
%%timeit

pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
               CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
               Cs_t.index('a couple of'),
               m = 200)

111 ms ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [230]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')
Ps_t.index('⋊.k.ɑ.n.t')

2773

9358

## Calculation volume organization for Case 1

See Section 6.2 for explanation of the "cases".

### Slices: Calculate results for groups of reconstructed wordforms by (input prefix, context)

In [231]:
len(Ps_t)

33059

In [232]:
def retrieveKcousins(r, maxK):
    word_cousin_indices = union([set(list(cousin_mats[k][Ps_t.index(r),:].nonzero()[0])) for k in range(maxK)])
    word_cousins = list(map(lambda w_idx: Ws_t[w_idx], word_cousin_indices))
    return word_cousins

In [233]:
retrieveKcousins('⋊.k.ɑ.n.t', 2)

['⋊.k.ɑ.ɹ.t.n.⋉',
 '⋊.k.ɑ.ɹ.t.u.n.⋉',
 '⋊.k.ɑ.ɹ.t.ɛ.k.s.⋉',
 '⋊.k.ɑ.ɹ.t.ɛ.l.⋉',
 '⋊.k.ɑ.ɹ.t.ɹ.ɪ.dʒ.⋉',
 '⋊.k.ɑ.ɹ.t.⋉',
 '⋊.k.aʊ.n.t.i.⋉',
 '⋊.k.aʊ.n.t.l.ə.s.⋉',
 '⋊.k.aʊ.n.t.ɚ.b.æ.l.ɪ.n.s.⋉',
 '⋊.k.aʊ.n.t.ɚ.f.ɪ.t.⋉',
 '⋊.k.aʊ.n.t.ɚ.æ.k.t.⋉',
 '⋊.k.aʊ.n.t.ɚ.⋉',
 '⋊.k.aʊ.n.t.⋉',
 '⋊.h.ɑ.n.t.⋉',
 '⋊.k.æ.n.t.n.⋉',
 '⋊.k.æ.n.t.ə.l.oʊ.p.⋉',
 '⋊.k.æ.n.t.ɚ.⋉',
 '⋊.k.æ.n.t.⋉',
 '⋊.dʒ.ɑ.n.t.⋉',
 '⋊.k.ʌ.n.t.ɹ.i.⋉',
 '⋊.k.ə.n.t.eɪ.n.⋉',
 '⋊.k.ə.n.t.æ.m.ə.n.eɪ.t.⋉',
 '⋊.k.ə.n.t.æ.m.ə.n.ɪ.n.t.⋉',
 '⋊.k.ə.n.t.ɛ.m.p.t.⋉',
 '⋊.k.ə.n.t.ɛ.n.d.⋉',
 '⋊.k.ə.n.t.ɛ.n.t.ɪ.d.⋉',
 '⋊.k.ə.n.t.ɛ.n.t.⋉',
 '⋊.w.ɑ.n.t.⋉',
 '⋊.k.ə.n.t.ɛ.n.tʃ.ɪ.n.⋉',
 '⋊.k.ə.n.t.ɪ.g.j.u.ə.s.⋉',
 '⋊.k.ə.n.t.ɪ.n.dʒ.ɪ.n.s.i.⋉',
 '⋊.k.ə.n.t.ɪ.n.j.u.eɪ.ʃ.ɪ.n.⋉',
 '⋊.k.ə.n.t.ɪ.n.j.u.ə.s.⋉',
 '⋊.k.ə.n.t.ɪ.n.j.u.⋉',
 '⋊.k.ə.n.t.ɹ.oʊ.l.⋉',
 '⋊.k.ə.n.t.ɹ.æ.k.ʃ.ɪ.n.⋉',
 '⋊.k.ə.n.t.ɹ.æ.p.ʃ.ɪ.n.⋉',
 '⋊.k.ə.n.t.ɹ.ɪ.b.j.u.t.ɚ.⋉',
 '⋊.k.ə.n.t.ɹ.ɪ.b.j.u.t.⋉',
 '⋊.k.ɑ.n.d.u.ɪ.t.⋉',
 '⋊.k.ɑ.n.d.ə.m.ɪ.n.i.ə.m.⋉',
 '⋊.k.ɑ.n.d.ɛ.m.n.eɪ.ʃ.ɪ.n.⋉',
 '⋊.k.ɑ.n.d

In [234]:
cousin_mats[0].shape
cousin_mats[0][Ps_t.index('⋊.k.ɑ.n.t'), Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')] #should be 1

(33059, 9172)

1

In [235]:
wordsWithPrefix('⋊.k.ɑ.n.t', Ws)

{'⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉',
 '⋊.k.ɑ.n.t.n.ɪ.n.t.⋉',
 '⋊.k.ɑ.n.t.æ.k.t.⋉',
 '⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉',
 '⋊.k.ɑ.n.t.ɛ.k.s.t.⋉',
 '⋊.k.ɑ.n.t.ɛ.s.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉'}

In [236]:
def get_nnz_word(my_k, j):
    return cousin_mats[my_k][:,j].nnz
    
cousin_mats[1].shape
word_nnz_counts_by_k = {my_k:par(delayed(get_nnz_word)(my_k, j) for j in range(cousin_mats[my_k].shape[1]))
                        for my_k in range(5)}

(33059, 9172)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0041s.) Setting batch_size=98.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 816 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 2090 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 3364 tasks      | elapsed:    0.6s
[Parallel(n_jobs=16)]: Done 4834 tasks      | elapsed:    0.8s
[Parallel(n_jobs=16)]: Done 9172 out of 9172 | elapsed:    1.3s finished
[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0124s.) Setting batch_size=32.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]:

[Parallel(n_jobs=16)]: Done 5608 tasks      | elapsed:   13.1s
[Parallel(n_jobs=16)]: Done 5920 tasks      | elapsed:   14.0s
[Parallel(n_jobs=16)]: Done 6248 tasks      | elapsed:   14.8s
[Parallel(n_jobs=16)]: Done 6576 tasks      | elapsed:   15.7s
[Parallel(n_jobs=16)]: Done 6920 tasks      | elapsed:   16.5s
[Parallel(n_jobs=16)]: Done 7264 tasks      | elapsed:   17.3s
[Parallel(n_jobs=16)]: Done 7624 tasks      | elapsed:   18.3s
[Parallel(n_jobs=16)]: Done 7984 tasks      | elapsed:   19.2s
[Parallel(n_jobs=16)]: Done 8360 tasks      | elapsed:   20.2s
[Parallel(n_jobs=16)]: Done 8736 tasks      | elapsed:   21.0s
[Parallel(n_jobs=16)]: Done 9172 out of 9172 | elapsed:   21.9s finished


In [237]:
from statistics import mean, median

In [238]:
for my_k in range(5):
    print(f"max number of {my_k}-cousins = {max(word_nnz_counts_by_k[my_k])}")
    print(f"median number of {my_k}-cousins = {median(word_nnz_counts_by_k[my_k])}")
    print(f"mean number of {my_k}-cousins = {mean(word_nnz_counts_by_k[my_k])}")
    print(' ')

max number of 0-cousins = 18
median number of 0-cousins = 7.0
mean number of 0-cousins = 7.692433493240297
 
max number of 1-cousins = 159
median number of 1-cousins = 106.0
mean number of 1-cousins = 105.34910597470562
 
max number of 2-cousins = 1436
median number of 2-cousins = 968.0
mean number of 2-cousins = 964.3620802442215
 
max number of 3-cousins = 4921
median number of 3-cousins = 3979.0
mean number of 3-cousins = 3961.463802878325
 
max number of 4-cousins = 8023
median number of 4-cousins = 6811.0
mean number of 4-cousins = 6452.722743131269
 


In [239]:
def get_nnz_prefix(my_k, i):
    return cousin_mats[my_k][i,:].nnz
    
cousin_mats[1].shape
prefix_nnz_counts_by_k = {my_k:par(delayed(get_nnz_prefix)(my_k, i) for i in range(cousin_mats[my_k].shape[0]))
                          for my_k in range(5)}

(33059, 9172)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0040s.) Setting batch_size=100.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Batch computation too fast (0.0197s.) Setting batch_size=2024.
[Parallel(n_jobs=16)]: Done 832 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 33059 out of 33059 | elapsed:    0.4s finished
[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0071s.) Setting batch_size=56.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Batch computation too fast (0.0136s.)

In [240]:
for my_k in range(5):
    print(f"max number of {my_k}-cousins = {max(prefix_nnz_counts_by_k[my_k])}")
    print(f"median number of {my_k}-cousins = {median(prefix_nnz_counts_by_k[my_k])}")
    print(f"mean number of {my_k}-cousins = {mean(prefix_nnz_counts_by_k[my_k])}")
    print(' ')

max number of 0-cousins = 9172
median number of 0-cousins = 1
mean number of 0-cousins = 2.134214586043135
 
max number of 1-cousins = 9171
median number of 1-cousins = 1
mean number of 1-cousins = 29.228409812759008
 
max number of 2-cousins = 9162
median number of 2-cousins = 11
mean number of 2-cousins = 267.55585468404973
 
max number of 3-cousins = 9010
median number of 3-cousins = 85
mean number of 3-cousins = 1099.0818234066367
 
max number of 4-cousins = 8774
median number of 4-cousins = 233
mean number of 4-cousins = 1790.265071538764
 


In [241]:
cousin_mats[1].shape

(33059, 9172)

In [242]:
def to_prefix_idx(w):
    return Ps_t.index(w)

prefix_indices_of_full_wordforms = par(delayed(to_prefix_idx)(w) for w in Ws)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0031s.) Setting batch_size=130.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Batch computation too fast (0.0508s.) Setting batch_size=1022.
[Parallel(n_jobs=16)]: Done 9172 out of 9172 | elapsed:    0.5s finished


In [243]:
def to_word_idx(p):
    return Ws_t.index(p)

def get_wordform_idx(prefix_idx):
    return to_word_idx(Ps_t[prefix_idx])

# get_wordform_idx = {prefix_idx:to_word_idx(Ps_t[prefix_idx])
#                     for prefix_idx in prefix_indices_of_full_wordforms}

In [244]:
Ps_t[8370]
get_wordform_idx(8370)
Ws_t[get_wordform_idx(8370)]

'⋊.k.oʊ.ɑ.p.ɚ.eɪ.t.⋉'

2511

'⋊.k.oʊ.ɑ.p.ɚ.eɪ.t.⋉'

In [245]:
cousin_mats[k].shape

(33059, 9172)

In [246]:
def getKcousin_idxs_of_wordform(w_idx, maxK, asType=None, onlyWordCousins = False, getPrefixIndices=True):
    kcousin_indices = union([set(list(cousin_mats[k][:,w_idx].nonzero()[0])) for k in range(maxK)])
    if onlyWordCousins:
    # top line is ≈100x slower
#         kcousin_indices = {prefix_idx for prefix_idx in kcousin_indices if prefix_idx in prefix_indices_of_full_wordforms}
        kcousin_indices = set.intersection(kcousin_indices, prefix_indices_of_full_wordforms)
    if not getPrefixIndices:
        kcousin_indices = set(map(get_wordform_idx, kcousin_indices))
    
    if asType == 'ndarray':
        kcousin_indices = np.array(sorted(list(kcousin_indices)), dtype='int32')
        return kcousin_indices
    elif asType == 'torch':
        kcousin_indices = torch.tensor(sorted(list(kcousin_indices)), dtype=torch.int32)
        return kcousin_indices
    else:
        return kcousin_indices
    
def getKcousin_idxs_of_prefix(p_idx, maxK, asType=None):
    kcousin_indices = union([set(list(cousin_mats[k][p_idx].nonzero()[0])) for k in range(maxK)])
    
    if asType == 'ndarray':
        kcousin_indices = np.array(sorted(list(kcousin_indices)), dtype='int32')
        return kcousin_indices
    elif asType == 'torch':
        kcousin_indices = torch.tensor(sorted(list(kcousin_indices)), dtype=torch.int32)
        return kcousin_indices
    else:
        return kcousin_indices

In [247]:
random_source_wordform
getKcousin_idxs_of_wordform(Ws_t.index(random_source_wordform), 2, 'ndarray', True, False)

'⋊.ɹ.ɪ.k.u.p.ɚ.eɪ.t.⋉'

array([8758, 8759, 8772], dtype=int32)

In [248]:
random_source_prefix
getKcousin_idxs_of_prefix(Ps_t.index(random_source_prefix), 2, 'ndarray')

'⋊.æ.k.t.ɪ.v.ə.t'

array([7052, 7053], dtype=int32)

In [249]:
def pXhat0fX0i_pxtn_by_rc(produced_prefix_idx, x0k_cm, c_idx, m = 50, asType='ndarray', parallel=False):
    kCousin_idxs = getKcousin_idxs_of_prefix(produced_prefix_idx, k, 'ndarray')
    
    if x0k_cm is None:
        x0k_cm = prefixTox0kCM(Ps_t[produced_prefix_idx])
    
    if parallel:
        estimates = par(delayed(pXhat0fX0i_pxtn)(w_idx, x0k_cm, c_idx, m=m) for w_idx in kCousin_idxs)
        estimates = np.array(list(estimates))
    else:
        estimates = np.array([pXhat0fX0i_pxtn(w_idx, x0k_cm, c_idx, m=m) for w_idx in kCousin_idxs])

    if asType == 'ndarray':
        return kCousin_idxs, estimates
    elif asType == 'torch':
#         return torch.tensor(kCousin_idxs), torch.tensor(estimates)
        return torch.from_numpy(kCousin_idxs), torch.from_numpy(estimates)
    else:
        return tuple(kCousin_idxs), tuple(estimates)

def pXhat0fX0i_pxtn_by_wc(produced_wordform_idx, x0k_cm, c_idx, m = 50, asType='ndarray', parallel=False):
    equivalent_prefix_idx = Ps_t.index(Ws_t[produced_wordform_idx])
    return pXhat0fX0i_pxtn_by_rc(equivalent_prefix_idx, x0k_cm, c_idx, m, asType, parallel)

In [250]:
random_source_prefix
random_source_prefix_idx = Ps_t.index(random_source_prefix)
random_source_prefix_x0kcm = prefixTox0kCM(random_source_prefix)

'⋊.æ.k.t.ɪ.v.ə.t'

In [251]:
print(f'x0k = {random_source_prefix}')
print(f'c = {random_context}')
cousin_idxs, post_probs = pXhat0fX0i_pxtn_by_rc(random_source_prefix_idx, random_source_prefix_x0kcm, random_context_idx)
print(f'({k}-cousin W-hat, p(W-hat|X0k = {random_source_prefix}, C = {random_context}))')
[(Ws_t[cousin_idx], post_probs[i]) for i, cousin_idx in enumerate(cousin_idxs)]

x0k = ⋊.æ.k.t.ɪ.v.ə.t
c = a couple of
(2-cousin W-hat, p(W-hat|X0k = ⋊.æ.k.t.ɪ.v.ə.t, C = a couple of))


[('⋊.æ.k.t.ɪ.v.eɪ.t.⋉', 0.00012125542060670379),
 ('⋊.æ.k.t.ɪ.v.ə.t.i.⋉', 0.63270417980207594)]

### Blocks: Calculate results for groups of input prefixes by context

In [ ]:
#FIXME

## Calculation volume organization 2

See Section 6.2 for explanation of the "cases".

In [ ]:
#FIXME

## Calculation volume organization 3

See Section 6.2 for explanation of the "cases".

### Slices: Calculate $\{p(\widehat{W} = w^* | W = w^*, C = c)\}$ for all $W$ and for a single given $c$ 

In [252]:
{w for w in Ws_t if ds2t(w)[0] != leftEdge or ds2t(w)[-1] != rightEdge}

set()

In [253]:
Ws_t.index('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉')
len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉')) #18

6304

18

In [ ]:
len(CMsByLengthByWordformIndex_torch)
CMsByLengthByWordformIndex_torch[18][6304].shape
CMsByLengthByWordformIndex_torch[18][6304]

In [254]:
len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ').shape

len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n').shape

len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉').shape

16

14

torch.Size([38, 14])

17

15

torch.Size([38, 15])

18

16

torch.Size([38, 16])

In [255]:
len(ds2t("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n"))
prefixTox0kCM("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n").shape
prefixTox0kCM("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n")

17

torch.Size([38, 15])

tensor([[0.0070, 0.0366, 0.0071, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0089, 0.0071, 0.0071, 0.0092, 0.0335, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0089, 0.0135, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0088, 0.0070, 0.0070, 0.0091, 0.0174, 0.0113, 0.0092, 0.0069,
         0.0090, 0.0157, 0.0088, 0.0071, 0.0070, 0.0142],
        [0.0068, 0.0086, 0.0068, 0.0068, 0.0088, 0.0169, 0.0110, 0.0089, 0.0068,
         0.0088, 0.0153, 0.0086, 0.0069, 0.0068, 0.0072],
        [0.0070, 0.0550, 0.0071, 0.2747, 0.0092, 0.1082, 0.0114, 0.0093, 0.0070,
         0.0091, 0.1078, 0.0089, 0.7089, 0.0071, 0.1108],
        [0.0068, 0.0087, 0.0069, 0.0069, 0.0090, 0.0171, 0.0111, 0.0090, 0.0068,
         0.0089, 0.0155, 0.0087, 0.0070, 0.0069, 0.0073],
        [0.0070, 0.0089, 0.

In [260]:
# CMsByLengthByWordformIndex_torch
X0f_CMs_by_wordform_index = [wordformTox0kCM(w, w_idx) for w_idx, w in enumerate(Ws_t)]
# X0f_CMs_by_wordform_index_torch = torch.tensor([wordformTox0kCM(w, w_idx) for w_idx, w in enumerate(Ws_t)])
# X0f_CMs_by_wordform_index_torch.shape

In [ ]:
def p_wstar_wstar_pxtn_by_wc(produced_wordform_idx, x0f_cm, c_idx, m = 50):
    return pXhat0fX0i_pxtn(produced_wordform_idx, x0f_cm, c_idx, m = m)

def p_wstar_wstar_pxtn_by_c(c_idx, m, asType='ndarray', parallel=True):
    if parallel:
#         estimates = par(delayed(pXhat0fX0i_pxtn)(w_idx, x0k_cm, c_idx, m=m) for w_idx in kCousin_idxs)
#         estimates = np.array(list(estimates))
# #         return kCousin_idxs, estimates
    else:
        estimates = np.array([p_wstar_wstar_pxtn_by_wc(w_idx, )])
#         estimates = np.array([pXhat0fX0i_pxtn(w_idx, x0k_cm, c_idx, m=m) for w_idx in kCousin_idxs])
# #         return kCousin_idxs, estimates
#     if asType == 'ndarray':
#         return kCousin_idxs, estimates
#     elif asType == 'torch':
# #         return torch.tensor(kCousin_idxs), torch.tensor(estimates)
#         return torch.from_numpy(kCousin_idxs), torch.from_numpy(estimates)
#     else:
#         return tuple(kCousin_idxs), tuple(estimates)
# #     return pXhat0fX0i_pxtn(w_idx, x0k_cm, c_idx, m=m)


In [ ]:
random_source_wordform
random_source_wordform_idx = Ws_t.index(random_source_wordform)
random_source_wordform_x0f_cm = wordformToX0fCM(random_source_wordform, random_source_wordform_idx)
random_source_wordform_x0f_cm.shape
random_source_wordform_x0f_cm

# Calculate distribution

## Number of calculations necessary under different calculation criteria

In general, the full distribution that could be calculated is $$p(\widehat{X}_0^f | X_0^k, C) = p(\widehat{W} | R, C)$$

In [ ]:
len(Ws)
len(Ps)
len(Cs)
"{0:,}".format( len(Ws) * len(Ps) * len(Cs) )
"{0:.2E}".format( len(Ws) * len(Ps) * len(Cs) )

For the chosen value of $k$, there are this many $(w,r)$ pairs to be calculated (for every $c$):

In [ ]:
# cousin_mats[k].shape
# sum([cousin_mats[k][i,:].nnz for i in range(len(Ps))])

In [ ]:
print(f'k = {k}')

cousin_mats[k].shape

def calc_count(i):
    return cousin_mats[k][i,:].nnz

num_seq_pairs_case1 = sum(list(par(delayed(calc_count)(i) for i in range(len(Ps)))))
num_calcs_case1 = len(Cs) * num_seq_pairs_case1

print('|(w,r)| pairs to do calculations for every c = {0:,}'.format( num_seq_pairs_case1 ))
print("|(w,r)| pairs there would be without the k-edit-distance-restriction = {0:,}".format( len(Ws) * len(Ps) ))
print(f'|Cs| = {len(Cs)}')
print('Est. of total number of calculations to do = {0:,} = {0:.2E}'.format(num_calcs_case1))

If $r$ is `False`, then the dimensions and number of calculations reduce to:

In [ ]:
len(Ws)
len(Ws)
len(Cs)
"{0:,}".format( len(Ws) * len(Ws) * len(Cs) )
"{0:.2E}".format( len(Ws) * len(Ws) * len(Cs) )

If $r$ is `False`, then for the chosen value of $k$, there are this many $(w',w)$ pairs to be calculated (for every $c$):

In [ ]:
print(f'k = {k}')

cousin_mats[k].shape

def calc_count(i):
    if Ps_t[i] in Ws:
        return cousin_mats[k][i,:].nnz
    else:
        return 0

num_seq_pairs_case2 = sum(list(par(delayed(calc_count)(i) for i in range(len(Ps)))))
num_calcs_case2 = len(Cs) * num_seq_pairs_case2

print("|(w',w)| pairs to do calculations for every c = {0:,}".format( num_seq_pairs_case2 ))
print("|(w',w)| pairs there would be without the k-edit-distance-restriction = {0:,}".format( len(Ws) * len(Ws) ))
print(f'|Cs| = {len(Cs)}')
print('Est. of total number of calculations to do = {0:,} = {0:.2E}'.format(num_calcs_case2))

If $r$ is `False` and $e$ is `True`, then the dimensions and number of calculations reduce to:

In [ ]:
len(Ws)
len(Cs)
num_calcs_case3 = len(Ws) * len(Cs)
"{0:,}".format( num_calcs_case3 )
"{0:.2E}".format( num_calcs_case3 )

## Determine dimensions and cells to be calculated vs. ignored

In [ ]:
assert (k != 0 and r) or (k != 0 and (not r) and (not e)) or (k == 0 and (not r) and e) or (k > 0 and (not r) and e)

### Option: Only calculate full source wordforms rather than all source prefixes?

In [ ]:
print(f"r = {r}")

if r:
    source_sequences = Ps_t
    print("Performing calculations for all source prefixes.")
else:
    source_sequences = Ws_t
    print("Performing calculations only for all complete source wordforms (and *not* all prefixes thereof).")

Ss_t = source_sequences    

print(f"|source_sequences| = {len(source_sequences)}")

### Suboption: Given $\neg r$ and a full source wordform $w^*$, only calculate posterior probability $p(\widehat{W} = w^* | W = w^*, C = c)$, $\forall w^*$?

In [ ]:
print(f"¬r ∧ e = {not r} ∧ {e} = {(not r) and e}")

if (not r) and e:
    print("Performing calculations only for those cases where the reconstructed wordform exactly matches the source wordform.")
    
else:
    print("Calculations not restricted only to exact reconstructed wordform matches.")

### Suboption: Restrict calculations only to (source sequence, reconstruction sequence) pairs within edit distance $k$?

In [ ]:
print(f"k = {k}")

if r or ((not r) and (not e)):
    print(f"Restricting calculations only to (source sequence, reconstructed sequence) pairs within edit distance {k}.")
elif (not r) and e:
    print(f"¬r ∧ e is equivalent to restricting calculations only to (source sequence, reconstructed sequence) pairs within edit distance 0.")
else:
    print("No restrictions on (source sequence, reconstructed sequence) pairs to be calculated.")

### Altogether...

In [ ]:
o

In [ ]:
if r and (0 < k <= 4):
    case = 1    
    myShape = (len(Ws), len(Ps), len(Cs))
    my_num_calcs = num_calcs_case1
    
    my_type = f'pW_RC_k{k}'
    status = '_unnormalized'

if (not r) and (0 < k <= 4) and (not e):
    case = 2
    myShape = (len(Ws), len(Ws), len(Cs))
    my_num_calcs = num_calcs_case2
    
    my_type = f'pW_WC_k{k}'
    status = '_unnormalized'
    
if (not r) and e:
    case = 3
#     myShape = (len(Ws), len(Ws), len(Cs))
    myShape = (len(Ws), len(Cs))
    my_num_calcs = num_calcs_case3

    my_type = f'pW_WC_e'
    status = ''

print(f'Case:\n\t{case}')
print(f'Shape of matrix:\n\t{myShape}')
print('Num elements:')
print("\t{0:,} = {0:.2E}".format(prod(myShape)))
print('Num calculations:')
print("\t{0:,} = {0:.2E}".format(my_num_calcs))
print('Est. size in GB of result:')
print("\t{0:,} = {0:.2E}".format(my_num_calcs * 8 / 1e9))
print("Density:")
print("\t{0:,}".format(my_num_calcs / prod(myShape)))    

print('Result suffix:')
my_fn_suffix = my_type + status
print(f"\t{my_fn_suffix}")

In [ ]:
abort_threshold_GB = 250
size_est_GB = (my_num_calcs * 8 / 1e9)
assert size_est_GB <= abort_threshold_GB, "Estimated size of resulting matrix is ≥ {1} GB:\n\tEst. size = {0:,}GB = {0:.2E}GB\n\tAborting calculation.".format(size_est_GB, abort_threshold_GB)

In [ ]:
# if exists(o + my_fn_suffix):
#     unnormalized_posterior = np.load(o + my_fn_suffix + '.npy',
#                                      allow_pickle = False)
# assert not exists(o + my_fn_suffix + '.npy'), 'Unnormalized distribution already found at filepath \n\t{0}'.format( o + my_fn_suffix + '.npy' )

In [ ]:
# unnormalized_posterior_torch = np.zeros(myShape)

## Case 1: $r \land (0 < k \leq 4)$

## Case 2: $\neg r \land (0 < k \leq 4) \land \neg e$

## Case 3: $\neg r \land e$

In [ ]:
myShape
blockShape = (myShape[0], 1) #every context is a slice
blockShape

In [ ]:
dom = tiledb.Domain(tiledb.Dim(name="segWord", domain=(0, myShape[0]-1), tile=blockShape[0], dtype=np.uint32),
                    tiledb.Dim(name="context", domain=(0, myShape[1]-1), tile=blockShape[1], dtype=np.uint32))

schema = tiledb.ArraySchema(domain=dom, sparse=False,
                            attrs=[tiledb.Attr(name="O", dtype=np.float64)])

In [ ]:
o
my_fn_suffix
o + '.' + my_fn_suffix

In [ ]:
array_name = o + '.' + my_fn_suffix
tiledb.SparseArray.create(array_name, schema)

In [ ]:
# rm -r 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC_e'